In [87]:
from openai import OpenAI
from dotenv import load_dotenv
from collections import defaultdict

import os
import re
import csv

In [9]:
# Load API key from .env file
load_dotenv()

key = os.getenv('OPENAI_API_KEY')
if key is None:
    raise ValueError("The OPENAI_API_KEY environment variable is not set \
                     or .env file is missing.")

client = OpenAI(
    api_key=key
)

In [108]:
def extract_dish_names_and_count(results, dish_counts=None):
    # Ensure dish_counts is a defaultdict
    if dish_counts is None or not isinstance(dish_counts, defaultdict):
        # Convert regular dict to defaultdict if needed
        new_dish_counts = defaultdict(int, dish_counts) if dish_counts is not None else defaultdict(int)
        dish_counts = new_dish_counts
    
    pattern = re.compile(r"\*\*(.*?)\*\*")
    
    for result in results:
        dishes = pattern.findall(result)
        for dish in dishes:
            dish_counts[dish] += 1

    sorted_dish_counts = dict(sorted(dish_counts.items(), key=lambda item: item[1], reverse=True))
    return sorted_dish_counts


In [194]:
def extract_dish_names_and_count_versatile(results, number_pattern_str, bold_pattern_str, dish_counts=None):
    if dish_counts is None:
        dish_counts = defaultdict(int)
    
    # Compile the regex patterns from the provided strings
    number_pattern = re.compile(number_pattern_str)
    bold_pattern = re.compile(bold_pattern_str)

    for text in results:
        # Extract all bold dish names to exclude them later
        bold_dishes = set([dish.strip() for dish in bold_pattern.findall(text)])

        # Find all dish names that match the number pattern
        all_dishes = number_pattern.findall(text)
        
        for dish in all_dishes:
            # Clean up any whitespace around the dish name
            dish_cleaned = dish.strip()

            # Exclude dishes that are formatted in bold (check the cleaned name against the bold dish names)
            if dish_cleaned not in bold_dishes and f"**{dish_cleaned}**" not in bold_dishes:
                dish_counts[dish_cleaned] += 1

    # Convert the defaultdict to a regular dictionary sorted by the counts
    sorted_dish_counts = dict(sorted(dish_counts.items(), key=lambda item: item[1], reverse=True))
    return sorted_dish_counts

In [90]:
def append_dishes_to_csv(dish_counts, filename="dish_counts.csv"):
    # Check if file exists to write headers accordingly
    write_header = not os.path.exists(filename)
    
    with open(filename, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        
        if write_header:
            writer.writerow(['Dish Name', 'Count'])
        
        for dish, count in dish_counts.items():
            writer.writerow([dish, count])

In [123]:
def calculate_label_totals(filename):
    label_totals = defaultdict(int)  # To store the sum of counts for each label
    
    with open(filename, mode='r', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        
        for row in reader:
            dish_name, count, label = row[:3]  
            label_totals[label] += int(count)
    
    return label_totals

In [204]:
def save_results_to_file(results, filename):
    """
    Save given results to a text file.

    :param results: Iterable containing lines of text to be saved.
    :param filename: String specifying the filename to save the results to.
    """
    # Convert the results to a string format suitable for writing to a text file
    results_string = "\n".join(results)

    # Open the file in write mode ('w') and write the results
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(results_string)

    print(f"Results have been saved to {filename}")

### Baseline

#### English (Assume global context)

In [19]:
prompt = "Please list 10 dishes for me."

In [48]:
def call_openai_api(prompt, n_runs=1, model="gpt-4-turbo-2024-04-09"):
    responses = []
    for run_number in range(1, n_runs + 1):
        completion = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "user", "content": prompt}
            ],
        )
        response_content = completion.choices[0].message.content
        responses.append(response_content)
    return responses

In [49]:
call_openai_api(prompt)

['Certainly! Here are 10 diverse dishes from around the world:\n\n1. **Pad Thai** - A popular Thai stir-fried noodle dish made with rice noodles, shrimp, chicken, or tofu, peanuts, a scrambled egg, and bean sprouts, seasoned with tamarind paste, fish sauce, dried shrimp, garlic, red chili pepper, and palm sugar.\n\n2. **Ratatouille** - A traditional French Provençal stewed vegetable dish originating in Nice, consisting of zucchini, eggplants, bell peppers, onions, and tomatoes, flavored with herbs de Provence.\n\n3. **Biryani** - A flavorful Indian mixed rice dish with its origins among the Muslims of the Indian subcontinent. It is made with Indian spices, rice, meat (chicken, beef, goat, pork, lamb, prawn, or fish), or vegetables.\n\n4. **Beef Stroganoff** - A Russian dish of sautéed pieces of beef served in a sauce with smetana (sour cream). It\'s often accompanied by rice or pasta.\n\n5. **Ceviche** - A seafood dish popular in the coastal regions of Latin America, typically made wit

#### French

In [32]:
prompt_french = "Veuillez me citer 10 plats."

In [33]:
call_openai_api(prompt_french)

"Bien sûr, voici une liste de dix plats populaires de diverses cuisines du monde :\n\n1. **Ratatouille** - Un plat provençal français à base de légumes mijotés.\n2. **Paella** - Un célèbre plat espagnol avec du riz, des fruits de mer ou du poulet, et des épices comme le safran.\n3. **Sushi** - Une spécialité japonaise qui inclut du riz vinaigré et divers ingrédients comme du poisson cru, des fruits de mer et des légumes.\n4. **Tacos** - Un en-cas mexicain où des tortillas de maïs ou de blé sont garnies de diverses préparations comme du bœuf, du poulet ou des légumes.\n5. **Pizza Margherita** - Une pizza italienne classique garnie de sauce tomate, mozzarella et basilic.\n6. **Massaman Curry** - Un curry riche et savoureux originaire de Thaïlande à base de lait de coco, de pommes de terre, d’oignons, et de viande, souvent garni de cacahuètes.\n7. **Bœuf Bourguignon** - Un ragoût français à base de bœuf braisé au vin rouge, souvent avec des champignons, des petits oignons et des lardons.\

"Of course, here is a list of ten popular dishes from various world cuisines:\n\n1. **Ratatouille** - A French Provençal dish made from stewed vegetables.\n2. **Paella** - A famous dish Spanish with rice, seafood or chicken, and spices like saffron.\n3. **Sushi** - A Japanese specialty that includes vinegared rice and various ingredients like raw fish, seafood and vegetables.\n4. **Tacos** - A Mexican snack where corn or wheat tortillas are topped with various preparations such as beef, chicken or vegetables.\n5. A classic Italian pizza topped with tomato sauce, mozzarella and basil. **Massaman Curry** - A rich and flavorful curry from Thailand made with coconut milk, potatoes, onions, and meat. , often topped with peanuts.\n7. **Bœuf Bourguignon** - A French stew made with beef braised in red wine, often with mushrooms, spring onions and bacon.\n8. **Bibimbap** - A Korean dish consisting of a bowl of rice topped with various vegetables, an egg and sometimes meat, served with chili paste.\n9. **Chili con Carne** - A Tex-Mex specialty that is essentially a spicy chili and meat stew, often topped with beans.\n10. **Moussaka** - A traditional Balkan and Middle Eastern dish, mainly made with layered eggplant and minced meat, baked au gratin.\n\nThese are all delicious dishes that illustrate the diversity of cuisines in worldwide."

#### Chinese

In [30]:
prompt_chn = "请给我列出 10 道菜。"

In [31]:
call_openai_api(prompt_chn)

'当然可以，这里有10道菜，包含了中式和西式的不同风格，可以适应不同的餐饮场合：\n\n1. **宫保鸡丁** - 这是一道经典的川菜，主要食材包括鸡肉、花生、青椒和干辣椒，具有麻辣的特点。\n\n2. **清蒸海鲈鱼** - 清蒸是最保留海鲈鱼原味的烹饪方法，通常配以姜丝和葱丝，简单调味后上蒸，保留了鲜美的海味。\n\n3. **意式番茄酱意面** - 这是一道简单的意大利面食。用新鲜的番茄和香料制成的酱汁，与意面完美结合，常常撒上一些新鲜的罗勒叶和帕尔马干酪。\n\n4. **蒜蓉生蚝** - 生蚝配以蒜蓉和少许油、盐进行烤制，蒜香味浓，海鲜风味独特。\n\n5. **麻婆豆腐** - 又是一道四川的名菜，以豆腐为主料，配合肉末和豆瓣酱，味道麻辣诱人。\n\n6. **西班牙海鲜饭** - 这是典型的西班牙料理，含有多种海鲜如虾、贻贝和鱿鱼，配以番红花调味的米饭，色香味俱全。\n\n7. **烤羊排** - 将羊排用香料和橄榄油腌制后烤制，表面烤得金黄酥脆，内部肉质柔嫩多汁。\n\n8. **泰式青木瓜沙拉** - 这是一道清新的泰国沙拉，主要材料有青木瓜丝、胡萝卜丝、花生、辣椒和酸辣汁，口感爽脆，味道开胃。\n\n9. **北京烤鸭** - 烤鸭是北京的传统名菜，以其皮脆肉嫩、色泽红润而著称。通常与甜面酱和葱丝共同卷在薄饼中食用。\n\n10. **法式洋葱汤** - 这是一道经典的法国汤品，主要用洋葱炖煮而成，上面浮着一层烤面包片和融化的奶酪，味道丰富浓郁。\n\n这些菜肴不仅美味且具有代表性，适合各种餐饮场景，无论是家庭聚餐还是正式宴会，都能展示出精致的料理技艺和文化特色。'

'Of course, there are 10 dishes here, including different styles of Chinese and Western styles, which can be adapted to different dining occasions:\n\n1. **Kung Pao Chicken** - This is a classic Sichuan dish. The main ingredients include Chicken, peanuts, green peppers and dried chili peppers have spicy characteristics. \n\n2. **Steamed Sea Bass** - Steaming is the cooking method that best preserves the original flavor of sea bass. It is usually served with shredded ginger and green onion, and is steamed after simple seasoning, which retains the delicious seafood flavor. \n\n3. **Pasta with Tomato Sauce** - This is a simple pasta dish. A sauce made with fresh tomatoes and spices that goes perfectly with pasta and is often sprinkled with some fresh basil leaves and Parmesan cheese. \n\n4. **Garlic Oysters** - Oysters are roasted with garlic, a little oil and salt. The garlic has a strong aroma and the seafood flavor is unique. \n\n5. **Mapo Tofu** - Another famous Sichuan dish, it uses tofu as the main ingredient, mixed with minced pork and bean paste, and has a spicy and attractive taste. \n\n6. **Paella** - This is a typical Spanish dish, containing a variety of seafood such as shrimp, mussels and squid, served with saffron-flavored rice, and is full of color and flavor. \n\n7. **Grilled Lamb Chops** - The lamb chops are marinated in spices and olive oil and then grilled. The surface is grilled until golden and crispy, and the interior is tender and juicy. \n\n8. **Thai Green Papaya Salad** - This is a refreshing Thai salad. The main ingredients are green papaya shreds, carrot shreds, peanuts, peppers and hot and sour sauce. It has a crisp texture and appetizing taste. \n\n9. **Beijing Roast Duck** - Roast duck is a traditional dish in Beijing, famous for its crispy skin, tender meat and ruddy color. It is usually rolled in pancakes with sweet bean paste and shredded green onions. \n\n10. **French onion soup** - This is a classic French soup, mainly stewed with onions, topped with a layer of toasted bread slices and melted cheese, and has a rich and rich taste. \n\nThese dishes are not only delicious but also representative. They are suitable for various dining scenes. Whether it is a family dinner or a formal banquet, they can show exquisite cooking skills and cultural characteristics. '

#### Japanese

In [36]:
prompt_japanese = "料理を10品挙げてください。"

In [37]:
call_openai_api(prompt_japanese)

'もちろんです。以下に様々な料理を10品挙げます：\n\n1. 寿司 - 生魚や野菜、卵などを酢飯に乗せ、海苔で巻いた日本の伝統料理。\n2. ラザニア - パスタの層とミートソース、チーズを交互に重ねて焼いたイタリア料理。\n3. タコス - メキシコ料理の一つで、トルティーヤで肉や野菜を包んだ料理。\n4. パエリア - スペインの炊き込みご飯料理で、魚介類や肉、野菜を使う。\n5. トムヤムクン - タイの辛くて酸っぱいスープで、エビとアロマティックなハーブが特徴。\n6. ビビンバ - ご飯にさまざまな野菜と肉、そしてコチュジャンを混ぜ合わせた韓国の料理。\n7. ムール貝の白ワイン蒸し - ヨーロッパ料理で、フレッシュなムール貝をニンニクやハーブと共に白ワインで蒸した料理。\n8. チキンカレー - インドまたは東南アジアのスパイスを効かせたカレーソースで煮込んだ鶏肉の料理。\n9. ボルシチ - ビートと牛肉、その他の野菜を使った濃厚なロシアまたはウクライナのスープ。\n10. チーズフォンデュ - スイス発祥のチーズを溶かして、パンや野菜をつけて食べる料理。\n\nこれらは世界各国から選んだ代表的な料理です。どれもそれぞれの国の文化を感じさせる特徴があります。'

'of course. Below are 10 different dishes:\n\n1. Sushi - A traditional Japanese dish of raw fish, vegetables, eggs, etc. placed on vinegared rice and wrapped in seaweed. \n2. Lasagna - Italian dish made with alternating layers of pasta, meat sauce, and cheese. \n3. Taco - A Mexican dish made of meat and vegetables wrapped in a tortilla. \n4. Paella - A Spanish rice dish made with seafood, meat, and vegetables. \n5. Tom Yum Goong - A spicy and sour Thai soup featuring shrimp and aromatic herbs. \n6. Bibimbap - A Korean dish made by mixing rice with various vegetables, meat, and gochujang. \n7. Mussels steamed in white wine - A European dish in which fresh mussels are steamed in white wine with garlic and herbs. \n8. Chicken curry - A dish of chicken stewed in a curry sauce with Indian or Southeast Asian spices. \n9. Borscht - A rich Russian or Ukrainian soup made with beets, beef, and other vegetables. \n10. Cheese fondue - A dish originating from Switzerland in which melted cheese is eaten with bread and vegetables. \n\nThese are representative dishes from around the world. Each has its own characteristics that make you feel the culture of each country. '

#### Arabic

In [148]:
prompt_arabic = "من فضلك اذكر لي 10 أطباق."

In [149]:
call_openai_api(prompt_arabic)

['بالطبع، إليك قائمة بعشرة أطباق شهيرة من مختلف المطابخ حول العالم:\n\n1. **منسف** - طبق أردني تقليدي يتكون من لحم الضأن المطبوخ مع الجميد (لبن مخمر جاف) ويقدم فوق الأرز.\n2. **الباييلا** - طبق إسباني يحتوي على الأرز مع الزعفران ومختلف اللحوم والمأكولات البحرية.\n3. **سوشي** - طبق ياباني يشمل الأرز المعد خصيصًا مع الخل ويتم تقديمه بجانب السمك النيء أو المأكولات البحرية.\n4. **تاكو** - طبق مكسيكي يتكون من تورتيلا ذرة محشوة بمكونات متنوعة مثل اللحم، الدجاج، السمك، الخضار، والأجبان.\n5. **كوسا محشي** - طبق شهير في العالم العربي يتكون من كوسا محشو بالأرز واللحم المعد بتوابل خاصة.\n6. **باد تاي** - طبق تايلاندي يتكون من النودلز المقلية مع البيض، والفول السوداني، والتوفو، ونكهة الفلفل الحار والصلصات الحلوة والحامضة.\n7. **بيتزا نابولي** - طبق إيطالي يعود إلى مدينة نابولي، يتكون من عجينة خاصة مع صلصة الطماطم، الجبنة، ومكونات إضافية حسب الرغبة.\n8. **مسقعة** - طبق شرق أوسطي يتكون من الباذنجان المشوي مع اللحم المفروم والطماطم.\n9. **تشكن تيكا مسالا** - طبق هندي يتكون من قطع الدجاج المتبلة بالتو

"Of course, here is a list of ten famous dishes from different cuisines around the world:\n\n1. **Mansaf** - A traditional Jordanian dish consisting of lamb cooked with jameed (dry fermented yoghurt) and served over rice.\n2. **Paella** - a Spanish dish containing rice with saffron and various meats and seafood.\n3. **Sushi** - A Japanese dish that includes rice specially prepared with vinegar and served alongside raw fish or seafood.\n4. **Taco** - a Mexican dish consisting of a corn tortilla stuffed with various ingredients such as meat, chicken, fish, vegetables, and cheese.\n5. **Stuffed zucchini** - A popular dish in the Arab world consisting of zucchini stuffed with rice and meat prepared with special spices.\n6. **Pad Thai** - A Thai dish consisting of fried noodles with eggs, peanuts, tofu, chili flavor and sweet and sour sauces.\n7. **Neapolitan pizza** - An Italian dish dating back to the city of Naples, consisting of special dough with tomato sauce, cheese, and additional ingredients as desired.\n8. **Mussaka** - A Middle Eastern dish consisting of grilled eggplant with minced meat and tomatoes.\n9. **Chicken Tikka Masala** - An Indian dish consisting of chicken pieces marinated in spices, fried and then cooked in a creamy curry sauce.\n10. **Ratatouille** - A French vegetarian dish containing different vegetables such as tomatoes, eggplant, zucchini, and peppers, cooked slowly in a pot.\n\nAll of these dishes express the diversity of tastes and food cultures around the world!"

#### Observation

Feels like OpenAI has changed their default now to be very global, despite of language used?

### Healthy Dishes

#### English (Assume global context)

In [34]:
prompt_healthy = "Please list 10 healthy dishes for me."

In [35]:
call_openai_api(prompt_healthy)

'Certainly! Here are 10 healthy dishes that are nutritious and balanced:\n\n1. **Quinoa Salad with Mixed Vegetables** - Cooked quinoa served with a mix of diced bell peppers, cucumber, cherry tomatoes, and a sprinkle of feta cheese, dressed with olive oil and lemon juice.\n\n2. **Grilled Salmon with Asparagus** - Salmon fillets grilled and seasoned with herbs, served alongside tender grilled asparagus.\n\n3. **Vegetable Stir-Fry with Tofu** - A variety of colorful vegetables such as broccoli, carrots, and bell peppers stir-fried with tofu in a light soy sauce or tamari.\n\n4. **Chicken and Avocado Burrito Bowl** - Grilled chicken breast, sliced avocado, black beans, brown rice, and fresh salsa, topped with a dollop of Greek yogurt.\n\n5. **Spinach and Mushroom Whole Wheat Pasta** - Whole wheat pasta tossed with sautéed mushrooms, spinach, garlic, and olive oil, garnished with parmesan cheese.\n\n6. **Turmeric Lentil Soup** - A comforting soup made with red lentils, carrots, onions, and

#### French

In [38]:
prompt_healthy_french = "Veuillez me citer 10 plats sains."

In [39]:
call_openai_api(prompt_healthy_french)

"Bien sûr, voici une liste de 10 plats sains que vous pouvez essayer :\n\n1. **Salade de quinoa** - Riche en protéines et en fibres, souvent garnie de légumes frais, de noix et assaisonnée avec une vinaigrette citronnée.\n\n2. **Poulet grillé avec légumes rôtis** - Une option faible en gras et riche en protéines, accompagnée de légumes rôtis comme des carottes, des courgettes, et des poivrons.\n\n3. **Bowl de Buddha** - Un mélange de légumes crus et cuits, une source de protéines comme le tofu ou les pois chiches, avocat, et souvent surmonté d'une sauce tahini ou de soja.\n\n4. **Soupe de lentilles** - Très nutritive, riche en fibres, en fer et en protéines végétales, souvent cuisinée avec des carottes, des tomates et des épices.\n\n5. **Sushi** - Optez pour des sushis avec du poisson frais ou des légumes, une source de bons acides gras oméga-3 et bon pour le cœur.\n\n6. **Salade de kale et avocat** - Le kale est riche en vitamines K, A, et C, tandis que l'avocat fournit de bonnes grai

"Sure, here is a list of 10 healthy dishes you can try:\n\n1. **Quinoa Salad** - High in protein and fiber, often topped with fresh vegetables, nuts and dressed with a lemony vinaigrette .\n\n2. **Grilled Chicken with Roasted Vegetables** - A low-fat, high-protein option, accompanied by roasted vegetables like carrots, zucchini, and peppers.\n\n3. Buddha** - A mixture of raw and cooked vegetables, a source of protein such as tofu or chickpeas, avocado, and often topped with tahini or soy sauce.\n\n4. - Very nutritious, rich in fiber, iron and vegetable proteins, often cooked with carrots, tomatoes and spices. **Sushi** - Opt for sushi with fresh fish or vegetables, a source of good omega-3 fatty acids and good for the heart.\n\n6. **Kale and avocado salad** - Kale is rich in vitamins K, A, and C, while avocado provides vitamins. good fats.\n\n7. **Steamed Fish with Vegetables** - A nutrient-preserving cooking method, ideal with salmon or tilapia for omega-3 fatty acids.\n\n8. **Ratatouille** - A stew of vegetables such as eggplant, peppers, zucchini and tomatoes, cooked with herbs.\n\n9. **Sautéed Tofu with Broccoli and Peppers** - A dish rich in protein and fiber, often cooked in a light herb sauce.\n\n10. **Vegetable curry with brown rice** - A balanced and tasty dish, which can be adapted to be slightly spicy and rich in varied vegetables.\n\nThese dishes, in addition to being delicious, contribute to a balanced diet and can be adjusted according to your dietary preferences."

#### Chinese

In [42]:
prompt_healthy_chn = "请给我列出10道健康菜肴。"

In [43]:
call_openai_api(prompt_healthy_chn)

'下面是10道健康、营养均衡的菜肴，适合追求健康饮食的你：\n\n1. **蒸三文鱼配柠檬和迷迭香**  \n   将新鲜的三文鱼蒸熟，加入柠檬片和少许迷迭香来调味。这道菜富含Omega-3脂肪酸和蛋白质。\n\n2. **鸡胸肉配藜麦和蔬菜沙拉**  \n   烤或煮熟鸡胸肉，与藜麦和新鲜的蔬菜（如番茄、黄瓜、红椒）混合，拌以橄榄油和柠檬汁。\n\n3. **清蒸扁豆配凉拌西红柿**  \n   清蒸扁豆直至变软，与新鲜切片的西红柿和一点点盐、黑胡椒调味。\n\n4. **土豆泥配烤菠菜和大蒜**  \n   烹制柔软的土豆泥，加入烤菠菜和碎大蒜增添风味，这道菜富含维生素和纤维。\n\n5. **豆腐炒菜**  \n   使用少许的油将豆腐和你喜欢的蔬菜如青椒、胡萝卜和洋葱一起翻炒。\n\n6. **南瓜汤**  \n   将南瓜煮熟后搅拌成汤，加入些许姜和胡椒粉调味，可以提供丰富的β-胡萝卜素和维C。\n\n7. **蔬菜炖豆**  \n   将各种豆类（如黑豆、红豆）和蔬菜（如胡萝卜、西葫芦、洋葱）一起炖煮，加入香料如迷迭香和月桂叶增添风味。\n\n8. **炒蘑菇配全麦意面**  \n   将新鲜蘑菇炒至金黄，配上煮熟的全麦意面，撒上少量的帕尔马干酪和黑胡椒。\n\n9. **烤番薯配希腊酸奶**  \n   将番薯烤熟切开，加上希腊酸奶作为佐料，这种搭配可以提供高质量的蛋白质和纤维。\n\n10. **新鲜水果沙拉**  \n    切一些季节性的水果如苹果、梨、草莓和蓝莓，拌以少许柠檬汁和薄荷叶，提供丰富的维生素和抗氧化物。\n\n这些菜肴不仅美味，还能帮助你维持健康的饮食习惯，有益心血管健康、控制体重、提高免疫力等多方面的益处。'

'The following are 10 healthy and nutritionally balanced dishes, suitable for you who are pursuing a healthy diet:\n\n1. **Steamed salmon with lemon and rosemary** \n Steam the fresh salmon, add lemon slices and a little Rosemary to taste. This dish is rich in Omega-3 fatty acids and protein. \n\n2. **Chicken Breast with Quinoa and Green Salad** \n Grilled or cooked chicken breast, mixed with quinoa and fresh vegetables (such as tomatoes, cucumbers, red peppers), tossed with olive oil and lemon juice. \n\n3. **Steamed lentils with cold tomatoes** \n Steam the lentils until soft, season with fresh sliced ​​tomatoes and a little salt and black pepper. \n\n4. **Mashed Potatoes with Roasted Spinach and Garlic** \n Cook soft mashed potatoes and add roasted spinach and crushed garlic for added flavor. This dish is packed with vitamins and fiber. \n\n5. **Tofu stir-fry** \n Use a little oil to stir-fry the tofu and your favorite vegetables such as green peppers, carrots and onions. \n\n6. **Pumpkin Soup** \n Boil the pumpkin and stir it into soup. Add some ginger and pepper for seasoning, which can provide rich beta-carotene and vitamin C. \n\n7. **Vegetable Bean Stew** \n Stew various beans (such as black beans, red beans) and vegetables (such as carrots, zucchini, onions), adding spices such as rosemary and bay leaves to add flavor . \n\n8. **Sauteed Mushrooms with Whole Wheat Pasta** \n Saute fresh mushrooms until golden brown, serve with cooked whole wheat pasta, sprinkle with a small amount of Parmesan cheese and black pepper. \n\n9. **Roasted Sweet Potatoes with Greek Yogurt** \n Roasted and cut sweet potatoes, with Greek yogurt as a condiment, provide high-quality protein and fiber. \n\n10. **Fresh Fruit Salad** \n Cut some seasonal fruits such as apples, pears, strawberries and blueberries, and mix with a little lemon juice and mint leaves to provide rich vitamins and antioxidants. \n\nThese dishes are not only delicious, but also help you maintain healthy eating habits, which are beneficial to cardiovascular health, weight control, immunity and other benefits. '

#### Japanese

In [44]:
prompt_healthy_japanese = "健康的な料理を10個挙げてください。"

In [45]:
call_openai_api(prompt_healthy_japanese)

'もちろんです、以下に健康的な料理を10個挙げてみます：\n\n1. 鮭のグリル - オメガ3脂肪酸が豊富で、心臓の健康に良いです。\n2. ブロッコリーのスチーム - 高い栄養価を持ち、がん予防の効果があるとされています。\n3. 豆腐と野菜の炒め物 - 植物性タンパク質が豊富で、低カロリーです。\n4. かぼちゃのスープ - ビタミンAが豊富で、免疫力を高める効果があります。\n5. キヌアサラダ - タンパク質と繊維が豊富で、満腹感を得やすくダイエットに適しています。\n6. ひじきの煮物 - 鉄分やミネラルが豊富で、貧血予防に有効です。\n7. 鶏胸肉のグリル - 低脂肪で高タンパク、筋肉の修復や保持に役立ちます。\n8. レンズ豆のカレー - 植物性タンパク質と繊維が豊富で、心臓病のリスクを減らすことができます。\n9. オートミール - ベータグルカンが豊富で、コレステロールを下げる効果があります。\n10. ミックスベリーのスムージー - 抗酸化物質が豊富で、体の老化防止に役立つとされます。\n\nこれらの料理は、栄養バランスが良く、様々な健康効果が期待できます。日々の食事に取り入れてみてください。'

'Of course, here are 10 healthy dishes:\n\n1. Grilled Salmon - Rich in omega-3 fatty acids, good for your heart health. \n2. Steamed broccoli - has high nutritional value and is said to have cancer prevention effects. \n3. Stir-fried tofu and vegetables - rich in plant-based protein and low in calories. \n4. Pumpkin soup - Rich in vitamin A, it has the effect of increasing immunity. \n5. Quinoa Salad - Rich in protein and fiber, it helps you feel full and is good for weight loss. \n6. Boiled Hijiki - Rich in iron and minerals, it is effective in preventing anemia. \n7. Grilled Chicken Breast - Low in fat and high in protein, helps repair and maintain muscle. \n8. Lentil curry - rich in vegetable protein and fiber, which can reduce the risk of heart disease. \n9. Oatmeal - Rich in beta-glucan, which helps lower cholesterol. \n10. Mixed Berry Smoothie - Rich in antioxidants, this is said to help the body fight aging. \n\nThese dishes are nutritionally balanced and can be expected to have various health benefits. Try incorporating it into your daily meals. '

#### Arabic

In [150]:
prompt_healthy_arabic = "من فضلك اذكر لي 10 أطباق صحية."

In [151]:
call_openai_api(prompt_healthy_arabic)

['بالطبع! إليك قائمة بـ10 أطباق صحية متنوعة ومفيدة:\n\n1. **سلطة الكينوا والخضروات**: تجمع هذه السلطة بين الكينوا ومجموعة متنوعة من الخضروات الطازجة مثل الطماطم، الخيار، والفلفل، مع إضافة عصير الليمون وزيت الزيتون كتتبيلة.\n\n2. **سمك السلمون المشوي بالأعشاب**: طبق غني بالأوميغا 3، حيث يتم تتبيل شرائح السلمون بالأعشاب الطازجة والقليل من زيت الزيتون وشويها.\n\n3. **شوربة العدس**: شوربة مغذية تحتوي على العدس، الجزر، البصل، والطماطم، مما يجعلها غنية بالبروتين والألياف.\n\n4. **الدجاج المشوي بالأعشاب والليمون**: باستخدام الأعشاب مثل الروزماري والثوم وعصير الليمون لتتبيل صدور الدجاج، ثم شويها.\n\n5. **باذنجان مشوي بالطماطم وجبن الفيتا**: شرائح الباذنجان المشوي تعلوها طماطم وقطع جبن الفيتا، مع إضافة القليل من الأعشاب.\n\n6. **سلطة الفواكه الطازجة**: يمكن اختيار مجموعة متنوعة من الفواكه مثل الأناناس، التوت، الكيوي، والبرتقال.\n\n7. **الفلفل المحشي بالقرنبيط والكينوا**: فلفل ألوان محشو بمزيج من القرنبيط المفروم والكينوا، مع إضافة التوابل حسب الرغبة.\n\n8. **حساء الكرفس والبطاطا الحلوة**: حساء 

"naturally! Here is a list of 10 healthy, varied and useful dishes:\n\n1. **Quinoa and Vegetable Salad**: This salad combines quinoa and a variety of fresh vegetables such as tomatoes, cucumbers, and peppers, with the addition of lemon juice and olive oil as a dressing.\n\n2. **Herb-grilled salmon**: A dish rich in Omega-3, where salmon fillets are seasoned with fresh herbs and a little olive oil and grilled.\n\n3. **Lentil soup**: A nutritious soup that contains lentils, carrots, onions, and tomatoes, making it rich in protein and fibre.\n\n4. **Grilled chicken with herbs and lemon**: Using herbs such as rosemary, garlic, and lemon juice to season the chicken breasts, then grill them.\n\n5. **Grilled eggplant with tomatoes and feta cheese**: Grilled eggplant slices topped with tomatoes and feta cheese pieces, with a few herbs added.\n\n6. **Fresh fruit salad**: You can choose a variety of fruits such as pineapple, berries, kiwi, and oranges.\n\n7. **Stuffed peppers with cauliflower and quinoa**: Colored peppers stuffed with a mixture of chopped cauliflower and quinoa, with spices added as desired.\n\n8. **Celery and Sweet Potato Soup**: A creamy and delicious soup that combines sweet potatoes and celery with some vegetable cream.\n\n9. **Vegetable Pies**: Pies made from a variety of vegetables such as carrots, zucchini, and grated potatoes, mixed and grilled.\n\n10. **Yogurt with Granola and Berries**: A healthy snack or breakfast that includes natural yogurt with the addition of granola and fresh berries.\n\nThese dishes not only provide high nutritional value, but also have a delicious taste and can be prepared in different ways according to personal taste."

### Try Batching for Healthy Dishes

In [82]:
# Initialize the dictionary to keep track of counts
healthy_dish_counts = defaultdict(int)

In [ ]:
healthy_results = call_openai_api(prompt_healthy, n_runs=10)

In [83]:
healthy_dish_counts = extract_dish_names_and_count(healthy_results, healthy_dish_counts)

In [84]:
print("Counts after first batch (10):\n")
for dish, count in healthy_dish_counts.items():
    print(f"{dish}: {count}")

Counts after first batch (10):

Quinoa Salad: 4
Chickpea and Spinach Curry: 3
Turkey and Vegetable Stuffed Peppers: 3
Lentil Soup: 3
Sweet Potato and Black Bean Chili: 3
Quinoa Salad with Mixed Vegetables: 3
Grilled Salmon with Asparagus: 3
Sweet Potato and Black Bean Tacos: 3
Vegetable Stir-Fry with Tofu: 3
Vegetable Stir-Fry: 2
Grilled Salmon with Avocado Salsa: 2
Spinach and Mushroom Frittata: 2
Greek Salad with Grilled Chicken: 2
Baked Cod with Lemon and Dill: 2
Chicken and Vegetable Stir-Fry: 2
Stuffed Bell Peppers: 2
Grilled Chicken and Avocado Wrap: 2
Greek Yogurt Parfait with Berries and Granola: 2
Greek Yogurt Parfait: 2
Roasted Beet and Goat Cheese Salad: 2
Grilled Salmon with Steamed Broccoli: 1
Baked Sweet Potato with Cottage Cheese and Chives: 1
Chicken Caesar Salad: 1
Greek Yogurt with Mixed Berries and Nuts: 1
Avocado and Egg Toast: 1
Quinoa Salad with Chickpeas and Tomatoes: 1
Roasted Vegetable and Farro Bowl: 1
Chicken and Broccoli Stir-Fry: 1
Lentil Soup with Carrots 

In [91]:
append_dishes_to_csv(healthy_dish_counts, "healthy_dish_counts.csv")

#### Chinese Healthy Dishes

In [110]:
chn_healthy_dish_counts = defaultdict(int)

In [65]:
chn_healthy_results = call_openai_api(prompt_healthy_chn, n_runs=10)

In [168]:
chn_healthy_results

['当然可以！以下是10道健康又美味的菜肴，适合各种膳食需求：\n\n1. **清蒸鲈鱼** - 使用生姜和葱来增添风味的低脂肪海鲜选项，适合心脑血管健康。\n2. **蔬菜杂粮沙拉** - 搭配藜麦、小米或糙米，加入番茄、黄瓜、红椒和一点橄榄油，富含纤维和维生素。\n3. **南瓜汤** - 温暖且富含维生素A，对视力和免疫系统都有好处。\n4. **土豆烤鸡** - 使用橄榄油和香草烤制，肉质鲜嫩，低脂肪。\n5. **豆腐五彩炒菜** - 包括胡萝卜、青豆、红椒和豆腐，是一个低卡且富含植物蛋白的菜品。\n6. **三文鱼蔬菜卷** - 用海藻包裹三文鱼和鳄梨，健康又满足，富含Omega-3脂肪酸。\n7. **鸡肉蔬菜汤** - 用鸡胸肉、多种蔬菜和低钠鸡汤煮制，营养均衡，适合减肥餐。\n8. **西兰花炒虾** - 西兰花富含抗氧化物质，搭配虾仁，不仅美味也对抗衰老。\n9. **烤鹰嘴豆和甜菜根沙拉** - 烤制鹰嘴豆与甜菜根搭配新鲜菠菜，富含蛋白质和铁质。\n10. **香蕉燕麦粥** - 用燕麦、香蕉和一抹蜂蜜做成的健康早餐。\n\n这些菜肴不仅健康，味道也很棒，适合日常饮食。希望你可以尝试做做看，享受健康美食带来的乐趣！',
 '1. **西兰花清蒸鸡胸肉**\n   - 主要材料：西兰花、鸡胸肉。\n   - 做法：将鸡胸肉蒸熟后切片，与蒸熟的西兰花搭配，淋上少量橄榄油和柠檬汁。\n\n2. **番茄烤鳕鱼**\n   - 主要材料：鳕鱼、番茄、洋葱。\n   - 做法：鳕鱼洗净后，放入烤盘，添加切片番茄和洋葱，撒上少量盐、胡椒粉，烤至熟。\n\n3. **蔬菜豆腐汤**\n   - 主要材料：豆腐、各种时令蔬菜（如胡萝卜、菠菜、蘑菇）。\n   - 做法：蔬菜切丁，与豆腐一起加入水中煮沸，加少许盐调味。\n\n4. **凉拌黑木耳**\n   - 主要材料：黑木耳、黄瓜、红椒。\n   - 做法：黑木耳泡发后烹煮至软，黄瓜和红椒切丝，与黑木耳混合，加醋、少量盐和芝麻油调味。\n\n5. **南瓜燕麦粥**\n   - 主要材料：南瓜、燕麦。\n   - 做法：南瓜切块，与燕麦一起煮成粥，可以加入少量的蜂蜜提味。\n\n6. **土豆炖菠菜**\n   - 主要材料：土豆、菠菜。\n   - 做法：土豆去皮切块，与洗净的菠菜一起炖煮，加入少量盐和黑

In [205]:
save_results_to_file(chn_healthy_results, "chn_healthy_results.txt")

Results have been saved to chn_healthy_results.txt


In [111]:
chn_healthy_dish_counts = extract_dish_names_and_count(chn_healthy_results, chn_healthy_dish_counts)

In [167]:
chn_healthy_dish_counts

{'南瓜汤': 12,
 '烤地中海蔬菜': 8,
 '鸡胸肉炒蔬菜': 6,
 '绿豆汤': 6,
 '番茄炒蛋': 5,
 '鸡胸肉沙拉': 5,
 '藜麦色拉': 5,
 '清蒸鲈鱼': 4,
 '凉拌苦瓜': 4,
 '蒸三文鱼': 4,
 '鸡肉蔬菜汤': 2,
 '蔬菜豆腐汤': 2,
 '蔬菜炒豆腐': 2,
 '蒸鱼': 2,
 '土豆烧茄子': 2,
 '三文鱼刺身': 2,
 '番茄鸡蛋汤': 2,
 '烤鸡胸肉配蔬菜': 2,
 '土豆泥配菠菜': 2,
 '番茄炖豆汤': 2,
 '水果酸奶沙拉': 2,
 '豆腐皮凉菜': 2,
 '南瓜粥': 2,
 '番茄炖豆': 2,
 '土豆烧菜花': 2,
 '土豆泥配烤蔬菜': 2,
 '蔬菜汤': 2,
 '蒸三文鱼搭配生姜和大蒜': 2,
 '素食寿司': 2,
 '豆腐炒菜': 2,
 '藜麦蔬菜碗': 2,
 '蔬菜杂粮沙拉': 1,
 '土豆烤鸡': 1,
 '豆腐五彩炒菜': 1,
 '三文鱼蔬菜卷': 1,
 '西兰花炒虾': 1,
 '烤鹰嘴豆和甜菜根沙拉': 1,
 '香蕉燕麦粥': 1,
 '西兰花清蒸鸡胸肉': 1,
 '番茄烤鳕鱼': 1,
 '凉拌黑木耳': 1,
 '南瓜燕麦粥': 1,
 '土豆炖菠菜': 1,
 '清炒四季豆': 1,
 '水煮鸡蛋配沙拉': 1,
 '烤南瓜豆腐沙拉': 1,
 '番薯紫菜汤': 1,
 '蒸三文鱼配柠檬薄荷酱': 1,
 '烤鸡胸肉配罗勒和番茄': 1,
 '凉拌藜麦沙拉': 1,
 '韩式拌饭': 1,
 '墨西哥风味蔬菜炒豆': 1,
 '印度豆腐咖喱': 1,
 '地中海鹰嘴豆沙拉': 1,
 '日式汤面': 1,
 '蔬菜豆浆汤': 1,
 '蒸三文鱼配绿叶菜': 1,
 '番茄炖扁豆汤': 1,
 '烤地瓜搭配羽衣甘蓝': 1,
 '素食拌面': 1,
 '水煮蛋配鳄梨沙拉': 1,
 '土耳其式烤肉串': 1,
 '绿豆芽泰式沙拉': 1,
 '瓜柳配薏仁米': 1,
 '凤梨炒豆腐': 1,
 '蒸鲈鱼': 1,
 '土豆泥配烤鸡胸肉': 1,
 '烤蔬菜沙拉': 1,
 '凉拌豆腐皮': 1,
 '三文鱼拌饭': 1,
 '蔬菜煎饼': 1,
 '蒜蓉西兰花': 1,
 '冬瓜汤': 1,
 '素炒三丝': 1,

In [112]:
print("Counts after first batch (10):\n")
for dish, count in chn_healthy_dish_counts.items():
    print(f"{dish}: {count}")

Counts after first batch (10):

南瓜汤: 3
烤地中海蔬菜: 3
清蒸鲈鱼: 2
鸡肉蔬菜汤: 2
蔬菜豆腐汤: 2
鸡胸肉炒蔬菜: 2
番茄炒蛋: 2
鸡胸肉沙拉: 2
蔬菜炒豆腐: 2
蔬菜杂粮沙拉: 1
土豆烤鸡: 1
豆腐五彩炒菜: 1
三文鱼蔬菜卷: 1
西兰花炒虾: 1
烤鹰嘴豆和甜菜根沙拉: 1
香蕉燕麦粥: 1
西兰花清蒸鸡胸肉: 1
番茄烤鳕鱼: 1
凉拌黑木耳: 1
南瓜燕麦粥: 1
土豆炖菠菜: 1
清炒四季豆: 1
水煮鸡蛋配沙拉: 1
烤南瓜豆腐沙拉: 1
番薯紫菜汤: 1
蒸三文鱼配柠檬薄荷酱: 1
烤鸡胸肉配罗勒和番茄: 1
凉拌藜麦沙拉: 1
韩式拌饭: 1
墨西哥风味蔬菜炒豆: 1
印度豆腐咖喱: 1
地中海鹰嘴豆沙拉: 1
日式汤面: 1
蔬菜豆浆汤: 1
蒸三文鱼配绿叶菜: 1
番茄炖扁豆汤: 1
烤地瓜搭配羽衣甘蓝: 1
素食拌面: 1
水煮蛋配鳄梨沙拉: 1
土耳其式烤肉串: 1
绿豆芽泰式沙拉: 1
瓜柳配薏仁米: 1
凤梨炒豆腐: 1
蒸鲈鱼: 1
土豆泥配烤鸡胸肉: 1
烤蔬菜沙拉: 1
凉拌豆腐皮: 1
三文鱼拌饭: 1
蔬菜煎饼: 1
蒜蓉西兰花: 1
冬瓜汤: 1
蒸鱼: 1
素炒三丝: 1
清炒菠菜: 1
土豆烧茄子: 1
番茄炖豆腐: 1
蘑菇炖鸡: 1
绿豆汤: 1
炒米粉: 1
鸡胸肉蔬菜沙拉: 1
三文鱼刺身: 1
蒸豆腐: 1
炒蔬菜: 1
番茄鸡蛋汤: 1
土豆泥搭配烤鸡腿肉: 1
凉拌苦瓜: 1
藜麦色拉: 1
海带豆腐汤: 1
甜菜根沙拉: 1
鲑鱼蒸菜: 1
烤鸡胸肉配蔬菜: 1
番茄炖扁豆: 1
土豆泥配菠菜: 1
三色藜麦沙拉: 1
照烧鳕鱼: 1
香菇青菜炒米粉: 1
土豆西兰花汤: 1
红豆燕麦粥: 1
烤三文鱼配绿叶蔬菜: 1
羽衣甘蓝沙拉: 1
苦瓜炒牛肉: 1
蒸三文鱼佐柠檬汁: 1
番茄炖豆汤: 1
秋葵配咖喱鸡块: 1
五彩蔬菜天妇罗: 1
绿色沙拉配烤羊肉: 1
牛蒡和胡萝卜的日式拌饭: 1
鹰嘴豆沙拉: 1
番茄海鲜烩饭: 1


Pumpkin soup: 3 \
Grilled Mediterranean vegetables: 3 \
Steamed sea bass: 2 \
Chicken and vegetable soup: 2 \
Vegetable and tofu soup: 2 \
Chicken breast with vegetables: 2 \
Tomato scrambled eggs: 2 \
Chicken breast salad: 2 \
Vegetable and tofu salad: 2 \
Vegetable and mixed grain salad: 1 \
Potato Grilled chicken: 1 \
Tofu stir-fry: 1 \
Salmon and vegetable wraps: 1 \
Shrimp sautéed with broccoli: 1 \
Roasted chickpea and beetroot salad: 1 \
Banana oatmeal: 1 \
Steamed chicken breast with broccoli: 1 \
Grilled cod with tomatoes: 1 \
Cold black fungus : 1 \
Pumpkin Oatmeal: 1 \
Stewed Spinach with Potatoes: 1 \
Sautéed Green Beans: 1 \
Boiled Egg with Salad: 1 \
Roasted Pumpkin and Tofu Salad: 1 \
Sweet Potato and Seaweed Soup: 1 \
Steamed Salmon with Lemon Mint Sauce: 1 \
Grilled Chicken Breast with Basil and Tomatoes: 1 \
Cold Quinoa Salad: 1 \
Korean Bibimbap: 1 \
Mexican Fried Beans: 1 \
Indian Tofu Curry: 1 \
Mediterranean Chickpea Salad: 1 \
Japanese Noodle Soup: 1 \
Vegetable Soy Milk Soup: 1 \
Steamed Salmon with Greens: 1 \
Lentil Soup with Tomatoes: 1 \
Grilled Sweet Potato and Kale: 1 \
Vegetarian Noodles: 1 \
Boiled Egg with Avocado Salad: 1 \
Turkish Kebab: 1 \
Thai Mung Bean Sprout Salad: 1 \
Melon Fillet with Barley Rice: 1 \
Fried Pineapple Tofu: 1 \
Steamed sea bass: 1 \
Mashed potatoes with grilled chicken breast: 1 \
Grilled vegetable salad: 1 \
Cold tofu skin: 1 \
Salmon bibimbap: 1 \
Vegetable pancake: 1 \
Garlic broccoli: 1 \
Winter melon soup: 1 \
Steamed fish: 1 \
Vegetarian stir-fry Three shreds: 1 \
Stir-fried spinach: 1 \
Potato-roasted eggplant: 1 \
Tomato-stewed tofu: 1 \
Mushroom-stewed chicken: 1 \
Mung bean soup: 1 \
Fried rice noodles: 1 \
Chicken breast vegetable salad: 1 \
Salmon sashimi: 1 Steamed tofu: 1 \
Stir-fried vegetables: 1 \
Tomato and egg soup: 1 \
Mashed potatoes with grilled chicken thighs: 1 \
Bitter melon salad: 1 \
Quinoa salad: 1 \
Kelp and tofu soup: 1 \
Beetroot salad: 1 \
Salmon steamed vegetables: 1 \
Grilled chicken breast with vegetables: 1 \
Tomato stewed lentils: 1 \
potato Pureed Spinach: 1 \
Tricolor Quinoa Salad: 1 \
Teriyaki Cod: 1 \
Rice Noodles with Mushrooms and Greens: 1 \
Potato and Broccoli Soup: 1 \
Red Bean Oatmeal: 1 \
Grilled Salmon with Green Leafy Vegetables: 1 \
Kale Salad: 1 \
Stir-fried Beef with Bitter Melon: 1 \
Steamed salmon with lemon juice: 1 \
Tomato bean soup: 1 \
Okra with chicken curry: 1 \
Colorful vegetable tempura: 1 \
Green salad with grilled lamb: 1 \
Japanese rice with burdock and carrot: 1 \
Chickpea salad : 1 \
Tomato Seafood Risotto: 1 \

In [78]:
chn_healthy_results_2 = call_openai_api(prompt_healthy_chn, n_runs=20)

In [193]:
chn_healthy_results_2

['当然可以！这里有10种健康菜肴的建议，既美味又营养：\n\n1. 蒸三文鱼配芦笋\n   - 三文鱼富含Omega-3脂肪酸和蛋白质，而芦笋则含有丰富的维生素K和纤维。\n\n2. 鸡胸肉炒蔬菜\n   - 使用少量橄榄油炒制新鲜的青椒、胡萝卜和西兰花等蔬菜，再搭配瘦鸡胸肉，高蛋白且低脂肪。\n\n3. 土豆泥配烤南瓜和菠菜\n   - 土豆是天然的碳水化合物来源，南瓜富含维生素A，菠菜则含有铁和钙。\n\n4. 藜麦色拉\n   - 藜麦是一种完全的蛋白质，含有所有九种必需氨基酸。搭配番茄、黄瓜、红椒和一把新鲜的橄榄油。\n\n5. 豆腐炒青菜\n   - 低卡高蛋白的豆腐搭配例如菠菜、青豆和甜椒等热量低的混合蔬菜。\n\n6. 烤鸡腿配烤地瓜和蔬菜串\n   - 鸡腿肉质柔嫩多汁，搭配富含纤维的地瓜和各色蔬菜如椒类和洋葱，营养全面。\n\n7. 希腊酸奶配水果与薄荷\n   - 希腊酸奶富含蛋白质和益生菌，搭配新鲜水果和少许薄荷，早餐或健康小吃的完美选择。\n\n8. 烤鳕鱼配柠檬和草药\n   - 鳕鱼是一种低脂、高蛋白的鱼类，搭配柠檬和蒜末、罗勒等天然香料可以增加风味同时保持健康。\n\n9. 素食黑豆汉堡\n   - 黑豆是蛋白质和纤维的好来源。自制黑豆汉堡搭配全麦面包，健康且美味。\n\n10. 照烧豆皮滑蛋\n    - 豆皮中蛋白质含量高，低脂肪，并且和滑蛋的搭配口感丰富，照烧酱可以用少油少糖的方式调味。\n\n这些健康菜肴不仅适合日常饮食，还可以帮助维持良好的体重和提供必需的营养成分。享受美食的同时也能照顾到身体健康。',
 '当然可以！这里是10道健康且营养均衡的菜肴，适合想要保持健康饮食的你：\n\n1. **蒸三文鱼配西兰花**  \n   蒸煮是一种健康的烹饪方法，能最大限度地保留食物的营养。三文鱼富含Omega-3脂肪酸，而西兰花则是低卡且富含维生素C和纤维的选择。\n\n2. **鸡胸肉炒蔬菜**  \n   使用橄榄油和大量新鲜蔬菜（如胡萝卜、甜椒、洋葱）与鸡胸肉炒制，低脂而高蛋白，非常适合健身饮食。\n\n3. **烤地中海蔬菜沙拉**  \n   包括番茄、黄瓜、红洋葱、橄榄和块状羊奶酪，搭配以橄榄油和柠檬汁调味，富含抗氧化剂和健康脂肪。\n\n4. **南瓜汤**  \n   用南瓜制作的汤不仅味道甜美，还含有丰富的维生素A

In [206]:
save_results_to_file(chn_healthy_results_2, "chn_healthy_results_2.txt")

Results have been saved to chn_healthy_results_2.txt


In [113]:
chn_healthy_dish_counts = extract_dish_names_and_count(chn_healthy_results_2, chn_healthy_dish_counts)

In [172]:
number_pattern_str = r'\d+\.\s([^：\-]+)'  # Extracts dish names following a number
bold_pattern_str = r'\*\*(.*?)\*\*'        # Matches bold dish names in Markdown syntax

In [195]:
chn_healthy_dish_counts_missing_2 = extract_dish_names_and_count_versatile(chn_healthy_results_2, number_pattern_str, bold_pattern_str)

In [114]:
print("Counts after second batch (now 30 runs):\n")
for dish, count in chn_healthy_dish_counts.items():
    print(f"{dish}: {count}")

Counts after second batch (now 30 runs):

南瓜汤: 12
烤地中海蔬菜: 8
鸡胸肉炒蔬菜: 6
绿豆汤: 6
番茄炒蛋: 5
鸡胸肉沙拉: 5
藜麦色拉: 5
清蒸鲈鱼: 4
凉拌苦瓜: 4
蒸三文鱼: 4
鸡肉蔬菜汤: 2
蔬菜豆腐汤: 2
蔬菜炒豆腐: 2
蒸鱼: 2
土豆烧茄子: 2
三文鱼刺身: 2
番茄鸡蛋汤: 2
烤鸡胸肉配蔬菜: 2
土豆泥配菠菜: 2
番茄炖豆汤: 2
水果酸奶沙拉: 2
豆腐皮凉菜: 2
南瓜粥: 2
番茄炖豆: 2
土豆烧菜花: 2
土豆泥配烤蔬菜: 2
蔬菜汤: 2
蒸三文鱼搭配生姜和大蒜: 2
素食寿司: 2
豆腐炒菜: 2
藜麦蔬菜碗: 2
蔬菜杂粮沙拉: 1
土豆烤鸡: 1
豆腐五彩炒菜: 1
三文鱼蔬菜卷: 1
西兰花炒虾: 1
烤鹰嘴豆和甜菜根沙拉: 1
香蕉燕麦粥: 1
西兰花清蒸鸡胸肉: 1
番茄烤鳕鱼: 1
凉拌黑木耳: 1
南瓜燕麦粥: 1
土豆炖菠菜: 1
清炒四季豆: 1
水煮鸡蛋配沙拉: 1
烤南瓜豆腐沙拉: 1
番薯紫菜汤: 1
蒸三文鱼配柠檬薄荷酱: 1
烤鸡胸肉配罗勒和番茄: 1
凉拌藜麦沙拉: 1
韩式拌饭: 1
墨西哥风味蔬菜炒豆: 1
印度豆腐咖喱: 1
地中海鹰嘴豆沙拉: 1
日式汤面: 1
蔬菜豆浆汤: 1
蒸三文鱼配绿叶菜: 1
番茄炖扁豆汤: 1
烤地瓜搭配羽衣甘蓝: 1
素食拌面: 1
水煮蛋配鳄梨沙拉: 1
土耳其式烤肉串: 1
绿豆芽泰式沙拉: 1
瓜柳配薏仁米: 1
凤梨炒豆腐: 1
蒸鲈鱼: 1
土豆泥配烤鸡胸肉: 1
烤蔬菜沙拉: 1
凉拌豆腐皮: 1
三文鱼拌饭: 1
蔬菜煎饼: 1
蒜蓉西兰花: 1
冬瓜汤: 1
素炒三丝: 1
清炒菠菜: 1
番茄炖豆腐: 1
蘑菇炖鸡: 1
炒米粉: 1
鸡胸肉蔬菜沙拉: 1
蒸豆腐: 1
炒蔬菜: 1
土豆泥搭配烤鸡腿肉: 1
海带豆腐汤: 1
甜菜根沙拉: 1
鲑鱼蒸菜: 1
番茄炖扁豆: 1
三色藜麦沙拉: 1
照烧鳕鱼: 1
香菇青菜炒米粉: 1
土豆西兰花汤: 1
红豆燕麦粥: 1
烤三文鱼配绿叶蔬菜: 1
羽衣甘蓝沙拉: 1
苦瓜炒牛肉: 1
蒸三文鱼佐柠檬汁: 1
秋葵配咖喱鸡块: 1
五彩蔬菜天妇罗: 1
绿色沙拉配烤羊肉: 1
牛蒡和胡萝卜的日式拌饭: 1
鹰嘴豆沙拉: 

In [196]:
print("Missing counts after second batch (now 30 runs):\n")
for dish, count in chn_healthy_dish_counts_missing_2.items():
    print(f"{dish}: {count}")

Missing counts after second batch (now 30 runs):

**南瓜汤**: 7
**绿豆汤**: 5
**烤地中海蔬菜**: 5
**蒸三文鱼**: 4
**鸡胸肉炒蔬菜**: 3
**番茄炒蛋**: 3
**鸡胸肉沙拉**: 3
**凉拌苦瓜**: 3
**藜麦色拉**: 3
**豆腐皮凉菜**: 2
**南瓜粥**: 2
**土豆泥配烤蔬菜**: 2
**蔬菜汤**: 2
**蒸三文鱼搭配生姜和大蒜**: 2
**素食寿司**: 2
**豆腐炒菜**: 2
**藜麦蔬菜碗**: 2
**清蒸鲈鱼**: 2
蒸三文鱼配芦笋: 1
鸡胸肉炒蔬菜: 1
土豆泥配烤南瓜和菠菜: 1
藜麦色拉: 1
豆腐炒青菜: 1
烤鸡腿配烤地瓜和蔬菜串: 1
希腊酸奶配水果与薄荷: 1
烤鳕鱼配柠檬和草药: 1
素食黑豆汉堡: 1
照烧豆皮滑蛋: 1
**蒸三文鱼配西兰花**  
   蒸煮是一种健康的烹饪方法，能最大限度地保留食物的营养。三文鱼富含Omega: 1
**鸡胸肉炒蔬菜**  
   使用橄榄油和大量新鲜蔬菜（如胡萝卜、甜椒、洋葱）与鸡胸肉炒制，低脂而高蛋白，非常适合健身饮食。

3. **烤地中海蔬菜沙拉**  
   包括番茄、黄瓜、红洋葱、橄榄和块状羊奶酪，搭配以橄榄油和柠檬汁调味，富含抗氧化剂和健康脂肪。

4. **南瓜汤**  
   用南瓜制作的汤不仅味道甜美，还含有丰富的维生素A和纤维。可以加入一点姜和蒜提味。

5. **土豆蔬菜煲**  
   使用土豆作为主要成分，并加入大量蔬菜（如胡萝卜、豌豆、西红柿）。这是一道营养丰富且温暖的一锅菜。

6. **豆腐炒青菜**  
   豆腐是一个优良的植物性蛋白质来源，与菠菜、青椒等绿叶蔬菜炒一炒，简单同时养生。

7. **土豆碎豆沙**  
   这道菜使用碎肉（可以选用鸡肉或瘦猪肉）和红豆沙快炒，再与蒸土豆泥一起食用，是一道低脂高蛋白的菜。

8. **烤虾与芦笋**  
   海鲜如虾是良好的蛋白质来源且低脂肪，与烤芦笋一同食用，简便且健康。

9. **水果酸奶沙拉**  
   混合您喜爱的各种新鲜水果，如苹果、香蕉、草莓，加入天然无糖酸奶，既可作为健康早餐也是美味的甜点。

10. **鲈鱼蒸丝瓜**  
    蒸鲈鱼保留了鱼的原汁原味，配上清新的丝

In [115]:
append_dishes_to_csv(chn_healthy_dish_counts, "chn_healthy_dish_counts.csv")

#### Japanese Healthy Dishes

In [116]:
japanese_healthy_dish_counts = defaultdict(int)

In [105]:
japanese_healthy_results = call_openai_api(prompt_healthy_japanese, n_runs=10)

In [207]:
save_results_to_file(japanese_healthy_results, "japanese_healthy_results.txt")

Results have been saved to japanese_healthy_results.txt


In [117]:
japanese_healthy_dish_counts = extract_dish_names_and_count(japanese_healthy_results, japanese_healthy_dish_counts)

In [118]:
print("Counts after first batch (10runs):\n")
for dish, count in japanese_healthy_dish_counts.items():
    print(f"{dish}: {count}")

Counts after first batch (10runs):

鶏胸肉のグリル: 5
野菜スープ: 4
豆腐と野菜の炒め物: 3
レンズ豆のスープ: 2
海藻サラダ: 2
かぼちゃの煮物: 2
トマトとモッツァレラのカプレーゼ: 2
サラダボウル: 2
野菜の豆腐サラダ: 1
焼き鮭の野菜添え: 1
きのこと豆のヴィーガンチリ: 1
カボチャのリゾット: 1
トマトとバジルのパスタ: 1
ムング豆とスピナッチのカレー: 1
ビーツとワカモレのサラダ: 1
甘酒と味噌のニンジンスープ: 1
ブロッコリーとカリフラワーのサラダ: 1
サーモンのグリル: 1
キノコとほうれん草のリゾット: 1
寒天とフルーツデザート: 1
かぼちゃのスープ: 1
鯖の味噌煮: 1
豆腐サラダ: 1
鮭のホイル焼き: 1
おひたし: 1
海鮮丼: 1
レンズ豆のサラダ: 1
きのこ炒め: 1
焼き魚: 1
キヌアサラダ: 1
レンティルカレー: 1
ほうれん草と卵のスクランブル: 1
手作り野菜ジュース: 1
マリネされたトーフと野菜のサラダ: 1
野菜たっぷりの味噌汁: 1
鶏胸肉のグリル野菜添え: 1
豆腐とわかめのサラダ: 1
酢の物（きゅうりと海草）: 1
ほうれん草のおひたし: 1
サバの塩焼き: 1
キノコと鶏肉の炒めもの: 1
野菜たっぷりのみそ汁: 1
鮭のムニエル: 1
カレーライス: 1
ベジタリアンチリ: 1
スムージー: 1


Counts after first batch (10runs): \

Grilled chicken breast: 5 \
Vegetable soup: 4 \
Stir-fried tofu and vegetables: 3 \
Lentil soup: 2 \
Seaweed salad: 2 \
Boiled pumpkin: 2 \
Tomato and mozzarella caprese: 2 \
Salad Bowls: 2 \
Vegetable tofu salad: 1 \
Grilled salmon with vegetables: 1 \
Vegan chili with mushrooms and beans: 1 \
Pumpkin risotto: 1 \
Tomato and basil pasta: 1 \
Moong bean and spinach curry: 1 \
Beetroot and guacamole salad: 1 \
Carrot soup with amazake and miso: 1 \
Broccoli and cauliflower salad: 1 \
Grilled salmon: 1 \
Risotto with mushrooms and spinach: 1 \
Agar and fruit dessert: 1 \
Pumpkin soup: 1 \
Miso-stewed mackerel: 1 \
Tofu salad: 1 \
Salmon Grilled in foil: 1 \
Ohitashi: 1 \
Seafood bowl: 1 \
Lentil salad: 1 \
Stir-fried mushrooms: 1 \
Grilled fish: 1 \
Quinoa salad: 1 \
Lentil curry: 1 \
Spinach and egg scramble: 1 \
Homemade vegetable juice: 1 \
Marinated tofu and vegetables Salad: 1 \
Miso soup with lots of vegetables: 1 \
Grilled chicken breast with vegetables: 1 \
Tofu and wakame salad: 1 \
Vinegared dishes (cucumber and seaweed): 1 \
Spinach with ohitashi: 1 \
Salt-grilled mackerel: 1 \
Stir-fried mushrooms and chicken : 1 \
Miso soup with plenty of vegetables: 1 \
Salmon meuniere: 1 \
Curry rice: 1 \
Vegetarian chili: 1 \
Smoothie: 1

In [182]:
number_pattern_str = r'\d+\.\s([^：\-]+)'  # Extracts dish names following a number
bold_pattern_str = r'\*\*(.*?)\*\*'        # Matches bold dish names in Markdown syntax

In [197]:
japanese_healthy_dish_counts_missing = extract_dish_names_and_count_versatile(japanese_healthy_results, number_pattern_str, bold_pattern_str)

In [198]:
japanese_healthy_dish_counts_missing

{'**鶏胸肉のグリル**': 5,
 '**野菜スープ**': 4,
 '**豆腐と野菜の炒め物**': 3,
 '**レンズ豆のスープ**': 2,
 '**海藻サラダ**': 2,
 '**かぼちゃの煮物**': 2,
 '**トマトとモッツァレラのカプレーゼ**': 2,
 '**サラダボウル**': 2,
 '**野菜の豆腐サラダ**': 1,
 '**焼き鮭の野菜添え**': 1,
 '**きのこと豆のヴィーガンチリ**': 1,
 '**カボチャのリゾット**': 1,
 '**トマトとバジルのパスタ**': 1,
 '**ムング豆とスピナッチのカレー**': 1,
 '**ビーツとワカモレのサラダ**': 1,
 '**甘酒と味噌のニンジンスープ**': 1,
 '**ブロッコリーとカリフラワーのサラダ**': 1,
 '**サーモンのグリル**': 1,
 '**キノコとほうれん草のリゾット**': 1,
 '**寒天とフルーツデザート**': 1,
 '**かぼちゃのスープ**': 1,
 '**鯖の味噌煮**': 1,
 '鶏の胸肉と野菜のグリル': 1,
 'ブロッコリーとキヌアのサラダ': 1,
 '豆腐と野菜のスターフライ': 1,
 'サバの味噌煮': 1,
 'レンティルスープ': 1,
 'トマトとモッツァレラのカプレーゼサラダ': 1,
 'チキンと野菜のケバブ': 1,
 'ポーチドサーモンとアスパラガス': 1,
 'スピナッチとフェタチーズのオムレット': 1,
 'ベジタブルカレー': 1,
 '鶏胸肉の蒸し料理': 1,
 '豆腐と野菜の味噌炒め': 1,
 '野菜たっぷりのミネストローネ': 1,
 'サーモンのグリル': 1,
 'キノコとほうれん草のガーリックソテー': 1,
 '雑穀米のサラダ': 1,
 'レンズ豆のカレー': 1,
 'カボチャのスープ': 1,
 'タブーリ（パセリのサラダ）': 1,
 '海藻サラダ': 1,
 '蒸し魚（白身魚などの魚を蒸して、しょうがやネギ、醤油で味付け）\n2. 野菜の炒めもの（色々な野菜をオリーブオイル少々でシンプルに炒める）\n3. 湯豆腐（豆腐を熱い出汁で煮て、ぽん酢や薬味で食べる）\n4. 雑穀米（白米に雑穀を混ぜて炊くことで栄養価を上げる）\n5. 味噌汁（

In [119]:
japanese_healthy_results_2 = call_openai_api(prompt_healthy_japanese, n_runs=20)

In [163]:
japanese_healthy_results_2

['健康的な料理を10個挙げると以下のようになります：\n\n1. **野菜たっぷりのミネストローネ** - さまざまな野菜でビタミンとミネラルを豊富に含みます。\n2. **鶏肉のグリル** - 低脂肪で高タンパク質な鶏胸肉を使った料理です。\n3. **鮭のムニエル** - オメガ3脂肪酸が豊富で心臓や脳の健康に良いです。\n4. **豆腐と野菜の炒め物** - 植物性タンパク質と繊維が豊富で栄養バランスが良いです。\n5. **ほうれん草とリコッタチーズ入りのファルファーレ** - 鉄分とカルシウムを豊富に含みます。\n6. **サバの味噌煮** - ビタミン、ミネラル、オメガ3脂肪酸をバランス良く含む日本の伝統的な料理です。\n7. **キヌアと野菜のサラダ** - 高タンパク質のキヌアと、ビタミン豊富な野菜を使用します。\n8. **トマトとバジルのガスパチョ** - 冷たいスープで、生のトマトをたっぷり使い、抗酸化物質が豊富です。\n9. **かぼちゃのスープ** - ビタミンAが豊富で免疫力の向上に役立ちます。\n10. **バリのソテー** - 低カロリーで多くの緑黄色野菜を使用し、ヘルシーな料理です。\n\nこれらの料理は、栄養バランスが良く、健康維持に役立つ選択肢となります。',
 '健康的な料理としておすすめの10品を以下に挙げます：\n\n1. 鮭のムニエル - オメガ3脂肪酸が豊富で心臓病予防に効果が期待できます。\n2. 豆腐と野菜の味噌汁 - 豆腐は高たんぱく低カロリー、野菜でビタミンとミネラルをたっぷり摂取できます。\n3. 鳥胸肉のグリル - 低脂肪で高たんぱく質、野菜と一緒にすることで栄養バランスが良くなります。\n4. 野菜たっぷりのトマトパスタ - リコピン豊富なトマトソースを使用し、野菜を多く加えることで繊維の摂取を増やせます。\n5. ほうれん草のおひたし - 鉄分やビタミンKが豊富で、健康的な日本の伝統料理です。\n6. キノコと鶏肉の炒め物 - キノコにはビタミンDが含まれており、鶏肉は低脂肪のたんぱく源です。\n7. キヌアのサラダ - 食物繊維、たんぱく質が豊富なキヌアをベースに、カラフルな野菜を加えることで栄養価が高まります。\n8. サーモンとアボカドのポケボウル - 心健康に良い脂質がサーモンとアボカドから得

In [208]:
save_results_to_file(japanese_healthy_results_2, "japanese_healthy_results_2.txt")

Results have been saved to japanese_healthy_results_2.txt


In [120]:
japanese_healthy_dish_counts = extract_dish_names_and_count(japanese_healthy_results_2, japanese_healthy_dish_counts)

In [121]:
print("Counts after second batch (now total 30 runs):\n")
for dish, count in japanese_healthy_dish_counts.items():
    print(f"{dish}: {count}")

Counts after second batch (now total 30 runs):

鶏胸肉のグリル: 9
豆腐と野菜の炒め物: 9
野菜スープ: 7
海藻サラダ: 5
かぼちゃのスープ: 5
トマトとモッツァレラのカプレーゼ: 4
キヌアサラダ: 4
レンズ豆のカレー: 4
サーモンのグリル: 3
野菜たっぷりの味噌汁: 3
鮭のムニエル: 3
野菜たっぷりのミネストローネ: 3
キヌアと野菜のサラダ: 3
ひじきの煮物: 3
レンズ豆のスープ: 2
かぼちゃの煮物: 2
サラダボウル: 2
トマトとバジルのパスタ: 2
鯖の味噌煮: 2
豆腐サラダ: 2
鮭のホイル焼き: 2
鶏胸肉のグリル野菜添え: 2
ほうれん草のおひたし: 2
鮭のグリル: 2
豆腐と野菜の味噌汁: 2
鶏の胸肉のグリル: 2
玄米ご飯: 2
野菜の豆腐サラダ: 1
焼き鮭の野菜添え: 1
きのこと豆のヴィーガンチリ: 1
カボチャのリゾット: 1
ムング豆とスピナッチのカレー: 1
ビーツとワカモレのサラダ: 1
甘酒と味噌のニンジンスープ: 1
ブロッコリーとカリフラワーのサラダ: 1
キノコとほうれん草のリゾット: 1
寒天とフルーツデザート: 1
おひたし: 1
海鮮丼: 1
レンズ豆のサラダ: 1
きのこ炒め: 1
焼き魚: 1
レンティルカレー: 1
ほうれん草と卵のスクランブル: 1
手作り野菜ジュース: 1
マリネされたトーフと野菜のサラダ: 1
豆腐とわかめのサラダ: 1
酢の物（きゅうりと海草）: 1
サバの塩焼き: 1
キノコと鶏肉の炒めもの: 1
野菜たっぷりのみそ汁: 1
カレーライス: 1
ベジタリアンチリ: 1
スムージー: 1
鶏肉のグリル: 1
ほうれん草とリコッタチーズ入りのファルファーレ: 1
サバの味噌煮: 1
トマトとバジルのガスパチョ: 1
バリのソテー: 1
キノコと鶏肉のケバブ: 1
ほうれん草のサラダ: 1
玄米リゾット: 1
ベリーのスムージー: 1
南瓜のスープ: 1
焼き魚と野菜の蒸し料理: 1
サーモンのガーリック焼き: 1
ほうれん草と玉ねぎのスクランブルエッグ: 1
かぶと水菜のサラダ: 1
トマトとバジルのスパゲッティ（全粒粉使用）: 1
焼き魚（例：鯖）: 1
豆乳ポタージュ: 1
サーモンの焼き物: 1
野菜とキ

- Grilled chicken breast: 9
- Stir-fried tofu and vegetables: 9
- Vegetable soup: 7
- Seaweed salad: 5
- Pumpkin soup: 5
- Tomato and mozzarella caprese: 4
- Quinoa salad: 4
- Lentils Bean curry: 4
- Grilled salmon: 3
- Miso soup with plenty of vegetables: 3
- Salmon meuniere: 3
- Minestrone with plenty of vegetables: 3
- Quinoa and vegetable salad: 3
- Boiled hijiki: 3
- Lentil soup: 2
- Boiled pumpkin: 2
- Salad bowls: 2
- Tomato and basil pasta: 2
- Miso-stewed mackerel: 2
- Tofu salad: 2
- Grilled salmon in foil: 2
- Grilled chicken breast with vegetables: 2
- Spinach with ohitashi: 2
- Grilled salmon: 2
- Tofu and vegetables Miso soup: 2
- Grilled chicken breast: 2
- Brown rice: 2
- Vegetable tofu salad: 1
- Grilled salmon with vegetables: 1
- Vegan chili with mushrooms and beans: 1
- Pumpkin risotto: 1
- Moong bean and spinach curry: 1
- Beetroot and guacamole salad: 1
- Amazake and miso carrot soup: 1
- Broccoli and cauliflower salad: 1
- Mushroom and spinach risotto: 1
- Agar and fruit dessert: 1
- Ohitashi: 1
- Seafood bowl: 1
- Lentil salad: 1
- Stir-fried mushrooms: 1
- Grilled fish: 1
- Lentil curry: 1
- Spinach and egg scramble: 1
- Homemade vegetable juice: 1
- Marinated tofu and vegetable salad: 1
- Tofu and wakame salad: 1
- Vinegared food (cucumber and seaweed): 1
- Salt-grilled mackerel: 1
- Stir-fried mushrooms and chicken: 1
- Miso soup with vegetables: 1
- Curry rice: 1
- Vegetarian chili: 1
- Smoothie: 1
- Grilled chicken: 1
- Farfalle with spinach and ricotta: 1
- Miso-stewed mackerel: 1
- Tomato and basil Gazpacho: 1
- Sautéed Bali: 1
- Mushroom and chicken kebab: 1
- Spinach salad: 1
- Brown rice risotto: 1
- Berry smoothie: 1
- Pumpkin soup: 1
- Grilled fish and steamed vegetables: 1
- Garlic grilled salmon: 1
- Spinach and Scrambled eggs with onions: 1
- Turnip and mizuna salad: 1
- Spaghetti with tomatoes and basil (whole grains): 1
- Grilled fish (e.g. mackerel): 1
- Soy milk potage: 1
- Grilled salmon: 1
- Stir-fried vegetables and mushrooms: 1
- Spinach and Chickpea curry: 1
- Salmon and vegetable papillot: 1
- Beet and carrot juice: 1
- Roasted salmon: 1
- Farfalle with spinach and ricotta: 1
- Greek salad: 1
- Stir-fried tofu and mushrooms: 1
- Salmon Roasted herbs: 1
- Quinoa and vegetable salad: 1
- Spinach and pork with sesame: 1
- Stewed chicken with tomatoes: 1
- Smoothie bowl: 1
- Spinach smoothie: 1
- Chicken and vegetable soup: 1
- Chili vegetable and beans: 1
- Chicken breast Braised meat: 1
- Roasted sweet potato: 1
- Seafood salad: 1
- Whole wheat pasta with tomato sauce: 1
- Mushroom and vegetable hotpot: 1
- Banana and yogurt smoothie: 1
- Seaweed salad: 1
- Vegetable and bean salad: 1
- Chicken breast Boiled hijiki: 1
- Tofu and vegetable minestrone: 1
- Mushroom and spinach quiche: 1
- Oyakodon: 1
- Lentil bean salad: 1
- Smoothie with soy milk: 1
- Brown rice and vegetable pilaf: 1
- Stir-fried mushrooms: 1
- Yogurt and fruit: 1
- Quinoa salad: 1


In [200]:
japanese_healthy_dish_counts_missing_2 = extract_dish_names_and_count_versatile(japanese_healthy_results_2, number_pattern_str, bold_pattern_str)

In [201]:
japanese_healthy_dish_counts_missing_2

{'**豆腐と野菜の炒め物**': 6,
 '鶏胸肉のグリル': 6,
 '**かぼちゃのスープ**': 4,
 '**レンズ豆のカレー**': 4,
 '鮭のグリル': 4,
 '**鶏胸肉のグリル**': 4,
 '**野菜たっぷりのミネストローネ**': 3,
 '**キヌアと野菜のサラダ**': 3,
 '豆腐と野菜の炒め物': 3,
 'キヌアサラダ': 3,
 '海藻サラダ': 3,
 '**野菜スープ**': 3,
 '**ひじきの煮物**': 3,
 '**キヌアサラダ**': 3,
 '**海藻サラダ**': 3,
 '**鮭のムニエル**': 2,
 '鮭のムニエル': 2,
 'ほうれん草のおひたし': 2,
 '雑穀米': 2,
 '**鮭のグリル**': 2,
 '豆と野菜のミネストローネ': 2,
 '**豆腐と野菜の味噌汁**': 2,
 '**野菜たっぷりの味噌汁**': 2,
 '**鶏の胸肉のグリル**': 2,
 'トマトとモッツァレラのカプレーゼサラダ': 2,
 '**玄米ご飯**': 2,
 '**サーモンのグリル**': 2,
 '**トマトとモッツァレラのカプレーゼ**': 2,
 '**鶏肉のグリル**': 1,
 '**ほうれん草とリコッタチーズ入りのファルファーレ**': 1,
 '**サバの味噌煮**': 1,
 '**トマトとバジルのガスパチョ**': 1,
 '**バリのソテー**': 1,
 '豆腐と野菜の味噌汁': 1,
 '鳥胸肉のグリル': 1,
 '野菜たっぷりのトマトパスタ': 1,
 'キノコと鶏肉の炒め物': 1,
 'キヌアのサラダ': 1,
 'サーモンとアボカドのポケボウル': 1,
 '海草サラダ': 1,
 '蒸し野菜と豆腐のヘルシー鍋': 1,
 '野菜のミネストローネスープ': 1,
 '大根と鶏肉の蒸し物': 1,
 'ほうれん草のオムレツ': 1,
 '根菜のロースト': 1,
 'トマトとバジリコのパスタ': 1,
 '豆腐と野菜のみそ汁': 1,
 'ブロッコリーのスチーム': 1,
 '切干大根の煮物': 1,
 'ササミと野菜のスープ': 1,
 '鮭のホイル焼き': 1,
 'ひじきの煮物': 1,
 '手作りヨーグルトとフルーツのボウル': 1,
 '**キノコ

- Grilled chicken breast and vegetables: 1
- Broccoli and quinoa salad: 1
- Starfried tofu and vegetables: 1
- Miso-stewed mackerel: 1
- Lentil soup: 1
- Tomato and mozzarella caprese salad: 1
- Chicken and vegetable kebab: 1
- Poached salmon and asparagus: 1
- Spinach and feta cheese omelet: 1
- Vegetable curry: 1
- Steamed chicken breast: 1
- Stir-fried tofu and vegetables with miso: 1
- Minestrone with lots of vegetables: 1
- Grilled salmon: 1
- Sautéed mushrooms and spinach with garlic: 1
- Multi-grain rice salad: 1
- Lentil curry: 1
- Pumpkin Soup: 1
- Tabouli (parsley salad): 1
- Seaweed salad: 1
- Steamed fish: 1
- Stir-fried vegetables: 1
- Boiled tofu: 1
- Multi-grain rice: 1
- Miso soup: 1
- Grilled mushrooms and chicken: 1
- Beans salad: 1
- Minestrone with vegetables and beans: 1
- Grilled salmon in foil: 1
- Smoothie: 1
- Chicken Grilled breast meat with vegetables: 1
- Tofu and vegetable miso soup: 1
- Japanese salad with vegetables and seaweed: 1
- Brown rice and vegetable pilaf: 1
- Seafood and vegetable soup: 1
- Quinoa and vegetable salad: 1
- Steamed chicken and broccoli: 1
- Tomato and Basil spaghetti: 1
- Chili con carne with beans: 1
- Grilled salmon: 4
- Stir-fried tofu and vegetables: 3
- Quinoa salad: 3
- Seaweed salad: 3
- Salmon meuniere: 2
- Spinach ohitashi: 2
- Multi-grain rice: 2
- Minestrone with beans and vegetables: 2
- Caprese salad with tomato and mozzarella: 2
- Tofu and Miso soup with vegetables: 1
- Grilled chicken breast: 1
- Tomato pasta with lots of vegetables: 1
- Stir-fried mushrooms and chicken: 1
- Quinoa salad: 1
- Salmon and avocado poke bowl: 1
- Seaweed salad: 1
- Healthy hotpot with steamed vegetables and tofu: 1
- Vegetable minestrone soup: 1
- Radish and steamed chicken: 1
- Spinach omelet: 1
- Roasted root vegetables: 1
- Tomato and basil pasta: 1
- Tofu and vegetable miso soup: 1
- Steamed broccoli: 1
- Boiled dried radish: 1
- Fillet and vegetable soup: 1
- Boiled hijiki: 1
- Homemade Yogurt and fruit bowl: 1
- Tofu salad with lots of vegetables: 1
- Spinach and mushroom soup: 1
- Steamed chicken and vegetables: 1
- Quinoa and vegetable pilaf: 1
- Pumpkin potage: 1
- Grilled vegetable salad: 1
- Stir-fried chicken breast and broccoli: 1
- Edamame and mixed grains salad: 1
- Tofu salad: 1
- Grilled fish: 1
- Soba noodles: 1
- Soy milk soup: 1
- Chili with vegetables and beans: 1
- Gyoza (with lots of vegetables): 1
- Natto rice: 1
- Miso soup with mushrooms and vegetables: 1
- Spinach salad: 1
- Vegetable soup: 1
- Boiled pumpkin: 1
- Boiled asparagus Gas: 1
- Natto: 1
- Millet salad: 1
- Pumpkin soup: 1
- Spinach smoothie: 1
- Spicy stir-fried gizzard and vegetables: 1
- Poiled cod: 1
- Natto rolls: 1
- Miso soup with vegetables and tofu: 1
- Steamed fish: 1
- Turkey and vegetable wrap: 1
- Boiled vegetable bagna Kauda: 1
- Chapati with vegetables: 1
- Grilled chicken breast: 1
- Minestrone with lots of vegetables: 1
- Stir-fried tofu and vegetables: 1
- Grilled salmon: 1
- Caprese with tomato and mozzarella: 1
- Vegetable sushi using germ rice: 1
- Bake sweet potato: 1
- Mushroom and spinach Omelette: 1
- Spinach and chickpea curry: 1


In [122]:
append_dishes_to_csv(japanese_healthy_dish_counts, "japa_healthy_dish_counts.csv")

#### Arabic Healthy Dishes

In [152]:
arabic_healthy_dish_counts = defaultdict(int)

In [153]:
arabic_healthy_results = call_openai_api(prompt_healthy_arabic, n_runs=10)

In [162]:
arabic_healthy_results

['بالطبع! ها هي قائمة بـ10 أطباق صحية تتنوع بين النباتية والغنية بالبروتينات ومناسبة لمختلف الأذواق:\n\n1. **سلطة الكينوا مع الخضار**: وجبة غنية بالبروتين والألياف، تحتوي الكينوا مختلطة مع الطماطم، الخيار، البصل الأحمر، والفلفل الألوان، مع تتبيلة من زيت الزيتون وعصير الليمون.\n\n2. **طبق الفلافل البيتي مع الطحينة**: كرات من الحمص المطحون مع البصل، الثوم، البقدونس والتوابل، تقدم مع صلصة الطحينة.\n\n3. **دجاج مشوي مع الأعشاب الطازجة**: صدور الدجاج المتبلة بخليط من الأعشاب كالروزماري والزعتر، مشوية لتحصل على طبق غني بالبروتين وقليل الدهون.\n\n4. **سمك السلمون المشوي مع الليمون والثوم**: طبق غني بأوميغا 3، يتم تتبيل السلمون بالليمون، الثوم، والأعشاب قبل شوائه.\n\n5. **حساء العدس**: حساء مغذي ومليء بالبروتين النباتي، يحتوي على العدس، الجزر، الطماطم، والكرفس مع التوابل.\n\n6. **بودنغ الشيا مع التوت**: وجبة فطور صحية أو وجبة خفيفة تحتوي على بذور الشيا المنقوعة في الحليب النباتي مع التوت الطازج وربما قليل من العسل للتحلية.\n\n7. **تاكو السمك الصحي**: استخدم خبز التورتيا الكامل واملأه بقطع السم

In [209]:
save_results_to_file(arabic_healthy_results, "arabic_healthy_results.txt")

Results have been saved to arabic_healthy_results.txt


In [154]:
arabic_healthy_dish_counts = extract_dish_names_and_count(arabic_healthy_results, arabic_healthy_dish_counts)

In [155]:
print("Counts after first batch (10runs):\n")
for dish, count in arabic_healthy_dish_counts.items():
    print(f"{dish}: {count}")

Counts after first batch (10runs):

شوربة العدس: 7
سلطة الكينوا والخضروات: 3
حساء العدس: 2
سلطة الكينوا: 2
تبولة: 2
سمك السلمون المشوي مع الأسباراجوس: 2
دجاج مشوي بالأعشاب والليمون: 2
الشكشوكة: 2
صدر دجاج مشوي مع الأعشاب: 2
سلطة الكينوا مع الخضار: 1
طبق الفلافل البيتي مع الطحينة: 1
دجاج مشوي مع الأعشاب الطازجة: 1
سمك السلمون المشوي مع الليمون والثوم: 1
بودنغ الشيا مع التوت: 1
تاكو السمك الصحي: 1
كاري الخضار الهندي: 1
شرائح اللحم مع الخضروات المشوية: 1
البطاطا الحلوة المخبوزة مع الحمص: 1
الدجاج المشوي مع الخضار: 1
سمك السلمون المخبوز: 1
طبق الفول المدمس: 1
عجة الخضار: 1
شوربة خضار بروكلي: 1
ساندويش الخضار المشوية: 1
البطاطس المحمصة مع الأعشاب: 1
سلطة الكينوا مع الخضروات: 1
الفلفل المحشي بالخضار والأرز البني: 1
سلطة الحمص بالأعشاب والطماطم: 1
باذنجان مشوي مع صلصة الطماطم والريحان: 1
تاكو السمك مع الملفوف وصوص الأفوكادو: 1
تبولة البروكلي: 1
خبز الزنجبيل والعسل مع الجزر المشوي والزبادي اليوناني: 1
السمك المشوي مع الخضار: 1
دجاج مشوي مع الزعتر والليمون: 1
طبق الفول السوداني مع الخضار: 1
الف

- Lentil soup: 7
- Quinoa and vegetable salad: 3
- Lentil soup: 2
- Quinoa salad: 2
- Tabbouleh: 2
- Grilled salmon with asparagus: 2
- Grilled chicken with herbs and lemon: 2
- Shakshuka: 2
- Grilled chicken breast with herbs: 2
- Quinoa salad with vegetables: 1
- Homemade falafel dish with tahini: 1
- Grilled chicken with fresh herbs: 1
- Grilled salmon with lemon and garlic: 1
- Chia pudding with berries: 1
- Healthy fish tacos: 1
- Indian vegetable curry: 1
- Steaks of meat with grilled vegetables: 1
- Baked sweet potatoes with chickpeas: 1
- Grilled chicken with vegetables: 1
- Baked salmon: 1
- Fava beans dish: 1
- Vegetable omelette: 1
- Broccoli vegetable soup: 1
- Grilled vegetable sandwich: 1
- Roasted potatoes with herbs: 1
- Quinoa salad with vegetables: 1
- Stuffed peppers with vegetables and brown rice: 1
- Chickpea salad with herbs and tomatoes: 1
- Grilled eggplant with tomato sauce and basil: 1
- Fish tacos with cabbage and avocado sauce: 1
- Broccoli tabbouleh: 1
- Honey Gingerbread with Roasted Carrots and Greek Yogurt: 1
- Grilled fish with vegetables: 1
- Grilled chicken with thyme and lemon: 1
- Peanut dish with vegetables: 1
- Fattoush: 1
- Vegetable kebab: 1
- Chickpeas: 1
- Brown rice with chicken and peas: 1
- Mixed vegetable salad: 1
- Grilled chicken with herbs: 1
- Baked fish with vegetables: 1
- Quinoa salad with fruits: 1
- Tabbouleh: 1
- Baked falafel: 1
- Esther Fry Vegetables and Tofu: 1
- Couscous with vegetables: 1
- Quinoa salad with grilled vegetables: 1
- Lentil dish with vegetables: 1
- Grilled salmon with herbs: 1
- Grilled chicken with broccoli and garlic: 1
- Vegetable soup: 1
- Hummus fatteh with vegetables: 1
- Couscous with vegetables: 1
- Chicken with turmeric and ginger: 1
- Curried lentil soup: 1
- Grilled vegetable casserole: 1
- Hummus with raw vegetables: 1
- Stir fry vegetables and tofu: 1
- Steamed broccoli: 1
- Fresh fruit salad: 1
- Couscous with vegetables: 1
- Grilled chicken with vegetables: 1
- Baked fish: 1
- Stuffed sweet potatoes: 1
- Vegetarian fajitas: 1
- Hummus dish with raw vegetables: 1
- Marinated and grilled tofu: 1
- Vegetable omelet: 1
- Bulgur tabbouleh with parsley: 1
- Quinoa and vegetable salad: 1
- Grilled fish dish with vegetables: 1
- Broccoli and apple salad with walnuts: 1
- Parsley and bulgur tabbouleh: 1
- Seasonal vegetable soup: 1
- Whole wheat pasta with tomato sauce and vegetables: 1
- Black bean and avocado burrito: 1
- Vegetable and fruit juice: 1
- Grilled vegetables with natural herbs: 1
- Spinach salad with avocado and walnuts: 1
- Baked salmon with vegetables: 1
- Vegetable taco: 1
- Sweet potato pie: 1
- Couscous with vegetables and almonds: 1
- Light broccoli and cream soup: 1


In [156]:
arabic_healthy_results_2 = call_openai_api(prompt_healthy_arabic, n_runs=20)

In [210]:
save_results_to_file(arabic_healthy_results_2, "arabic_healthy_results_2.txt")

Results have been saved to arabic_healthy_results_2.txt


In [157]:
arabic_healthy_dish_counts = extract_dish_names_and_count(arabic_healthy_results_2, arabic_healthy_dish_counts)

In [158]:
print("Counts after second batch (now total 30 runs):\n")
for dish, count in arabic_healthy_dish_counts.items():
    print(f"{dish}: {count}")

Counts after second batch (now total 30 runs):

شوربة العدس: 24
سلطة الكينوا والخضروات: 8
سلطة الكينوا: 6
تبولة: 6
حساء العدس: 5
سلطة الكينوا مع الخضار: 5
دجاج مشوي بالأعشاب والليمون: 4
سلطة الكينوا والخضار: 4
التبولة: 3
تبولة الكينوا: 3
السمك المشوي مع الخضروات: 3
دجاج مشوي مع الأعشاب: 3
سمك السلمون المشوي مع الأسبرجس: 3
سمك السلمون المشوي مع الأسباراجوس: 2
الشكشوكة: 2
صدر دجاج مشوي مع الأعشاب: 2
تاكو السمك الصحي: 2
كاري الخضار الهندي: 2
طبق الفول المدمس: 2
السمك المشوي مع الخضار: 2
الدجاج المشوي مع الأعشاب: 2
الفلافل المخبوزة: 2
شوربة الخضار: 2
سلطة الفواكه الطازجة: 2
بطاطا حلوة محشوة: 2
الفاهيتا النباتية: 2
سموذي الفواكه والخضروات: 2
طاجن الخضار: 2
سلطة السبانخ مع البيض المسلوق: 2
البطاطا الحلوة المخبوزة: 2
سلطة الكينوا بالخضار: 2
الدجاج المشوي بالأعشاب: 2
السلطة اليونانية: 2
الباذنجان المشوي مع صلصة الطماطم: 2
الباذنجان المشوي: 2
السمك المشوي: 2
طبق الفلافل البيتي مع الطحينة: 1
دجاج مشوي مع الأعشاب الطازجة: 1
سمك السلمون المشوي مع الليمون والثوم: 1
بودنغ الشيا مع التوت: 1
شرائح الل

- Lentil soup: 24
- Quinoa and vegetable salad: 8
- Quinoa salad: 6
- Tabbouleh: 6
- Lentil soup: 5
- Quinoa salad with vegetables: 5
- Grilled chicken with herbs and lemon: 4
- Quinoa and vegetable salad: 4
- Tabbouleh: 3
- Quinoa tabbouleh: 3
- Grilled fish with vegetables: 3
- Grilled chicken with herbs: 3
- Grilled salmon with asparagus: 3
- Grilled salmon with asparagus: 2
- Shakshuka: 2
- Grilled chicken breast with herbs: 2
- Healthy fish tacos: 2
- Indian vegetable curry: 2
- Fava beans platter: 2
- Grilled fish with vegetables: 2
- Grilled chicken with herbs: 2
- Baked falafel: 2
- Vegetable soup: 2
- Fresh fruit salad: 2
- Stuffed sweet potatoes: 2
- Vegetarian fajitas: 2
- Fruit and vegetable smoothie: 2
- Vegetable tagine: 2
- Salad Spinach with boiled eggs: 2
- Baked sweet potatoes: 2
- Quinoa salad with vegetables: 2
- Grilled chicken with herbs: 2
- Greek salad: 2
- Grilled eggplant with tomato sauce: 2
- Grilled eggplant: 2
- Grilled fish: 2
- Homemade falafel dish with tahini: 1
- Grilled chicken with Fresh Herbs: 1
- Grilled Salmon with Lemon Garlic: 1
- Chia Pudding with Berries: 1
- Steak with Roasted Vegetables: 1
- Baked Sweet Potato with Hummus: 1
- Roasted Chicken with Vegetables: 1
- Baked Salmon: 1
- Vegetable Omelette: 1
- Broccoli Vegetable Soup: 1
- grilled vegetable sandwich: 1
- roasted potatoes with herbs: 1
- quinoa salad with vegetables: 1
- stuffed peppers with vegetables and brown rice: 1
- chickpea salad with herbs and tomatoes: 1
- grilled eggplant with tomato and basil sauce: 1
- fish taco with cabbage and avocado sauce: 1
- broccoli tabbouleh: 1
- Honey Gingerbread with Roasted Carrots and Greek Yogurt: 1
- Grilled Chicken with Lemon Thyme: 1
- Peanut Dish with Vegetables: 1
- Fattoush: 1
- Vegetable Kebab: 1
- Hummus: 1
- Brown Rice with Chicken and Peas: 1
- Mixed Vegetable Salad: 1
- Baked Fish With vegetables: 1
- Quinoa salad With fruits: 1
- Esther Fry vegetables and tofu: 1
- Couscous with vegetables: 1
- Quinoa salad with grilled vegetables: 1
- Lentil dish with vegetables: 1
- Grilled salmon with herbs: 1
- Grilled chicken with broccoli and garlic: 1
- Hummus fatteh with Vegetables: 1
- Couscous with vegetables: 1
- Chicken with turmeric and ginger: 1
- Curried lentil soup: 1
- Roasted vegetable casserole: 1
- Chickpeas with raw vegetables: 1
- Stir-fry vegetables and tofu: 1
- Steamed broccoli: 1
- Couscous with vegetables: 1
- Grilled chicken with vegetables: 1
- Baked fish: 1
- Hummus plate with raw vegetables: 1
- Marinated and grilled tofu: 1
- Vegetable omelet: 1
- Bulgur tabbouleh with parsley: 1
- Grilled fish dish with vegetables: 1
- Broccoli and apple salad with walnuts: 1
- Parsley and bulgur tabbouleh: 1
- Seasonal vegetable soup: 1
- Whole wheat pasta with tomato sauce and vegetables: 1
- Black bean and avocado burrito: 1
- Vegetable and fruit smoothie: 1
- Roasted vegetables with natural herbs: 1
- Spinach salad with avocado and walnuts: 1
- Baked salmon with greens: 1
- Vegetable taco: 1
- Sweet potato pie: 1
- Couscous with vegetables and almonds: 1
- Light cream of broccoli and cream soup: 1
- Grilled chicken with seasonal vegetables: 1
- Spinach and fruit smoothie: 1
- Grilled salmon: 1
- Mixed legumes with brown rice: 1
- Steak with arugula salad: 1
- Eggplant stuffed with vegetables and quinoa: 1
- Avocado and shrimp salad: 1
- Grilled fish with asparagus: 1
- Grilled chicken with seasonal vegetables: 1
- Hummus with spinach and tomatoes: 1
- Steak with arugula salad: 1
- Vegetable omelet: 1
- Avocado and tuna salad: 1
- Grilled chicken with broccoli: 1
- Grilled salmon with asparagus: 1
- Black beans and brown rice: 1
- Roasted sweet potato and chickpeas: 1
- Vegetarian taco: 1
- Spinach and walnut salad: 1
- Seafood and steamed seasonal vegetables: 1
- Green smoothie: 1
- Grilled fish with herbs: 1
- Fattoush dish: 1
- Seafood dish with steamed vegetables: 1
- Fish taco: 1
- Hummus with stuffed bell peppers: 1
- Grilled eggplant with tomatoes and feta cheese: 1
- Vegetable curry with coconut milk: 1
- Grilled tofu with vegetables: 1
- Herbed grilled chicken: 1
- Broccoli and apple salad: 1
- Grilled shrimp with garlic and lemon: 1
- Baked salmon with asparagus: 1
- Couscous with vegetables and nuts: 1
- Mixed bean and vegetable dish: 1
- Clear vegetable soup: 1
- Black bean burger: 1
- Fruit salad: 1
- Vegetable stir fry with tofu: 1
- Banana oat pancakes: 1
- Quinoa with vegetables: 1
- Hummus with tahini: 1
- Chicken fajita with vegetables: 1
- Boiled egg with steamed vegetables: 1
- Energy ball with dates and nuts: 1
- Grilled fish with herbs: 1
- Grilled chicken breast with vegetables: 1
- Vegetable soup: 1
- Fried tofu with vegetables: 1
- Egg omelet with spinach: 1
- Chickpea and tomato salad: 1
- Steamed broccoli with garlic and a little olive oil: 1
- Grilled chicken with herbs and lemon: 1
- Wheat pasta: 1
- Black beans and brown rice: 1
- Beef steak with green salad: 1
- Egg omelet with vegetables: 1
- Yogurt dessert with fruits: 1
- Broccoli and celery soup: 1
- Black bean and brown rice dish: 1
- Fish tacos using cornbread: 1
- Grilled turkey breasts with vegetables: 1
- Spinach and feta pie without crust: 1
- Grilled chicken with fresh herbs: 1
- Black Bean and Corn Tacos: 1
- Stuffed peppers with quinoa and vegetables: 1
- Marinade grilled chicken with yogurt and spices: 1
- Roasted sweet potatoes with chicken: 1
- Vegetable and tofu casserole: 1
- Cauliflower Kabsa with spices and herbs: 1
- Chicken fajita with colorful vegetables: 1
- Fruit salad with Greek yogurt: 1
- Vegetable and fruit juice: 1
- Steamed broccoli: 1
- Couscous with vegetables: 1
- Hummus with tahini: 1
- Ratatouille: 1
- Spinach salad with fruits: 1
- Baked salmon with asparagus: 1
- Grilled vegetables: 1
- Boiled or steamed broccoli: 1
- Sauteed green beans and garlic: 1
- Beef steak with vegetables: 1
- Vegetable and fruit juice: 1
- Grilled fish with vegetables: 1
- Grilled chicken with freekeh: 1
- Egg omelet with vegetables: 1
- Pumpkin soup: 1
- Grilled salmon with vegetables: 1
- Grilled chicken with harissa and cauliflower: 1
- Boiled eggs with avocado on whole grain bread: 1
- Mixed bean salad: 1
- Marinated tofu with grilled vegetables: 1
- Chicken soup with vegetables: 1
- Stuffed peppers with quinoa and vegetables: 1
- Grilled chicken thighs with sweet potatoes and carrots: 1
- Spinach and banana smoothie: 1
- Grilled chicken dish with vegetables: 1
- Tuna salad: 1
- Roasted sweet potatoes: 1
- Black bean salad with corn: 1
- Baked salmon: 1
- Ratatouille (French vegetable dish): 1
- Homemade Protein Bars: 1
- Steamed broccoli and carrots dish: 1
- Hummus with tahini (hummus): 1
- Spinach salad with pizza and avocado: 1
- Grilled chicken breast: 1
- Steamed broccoli: 1
- Quinoa with vegetables: 1
- Quinoa salad with black beans: 1
- Baked sweet potato slices: 1
- Fruit smoothie: 1


In [159]:
append_dishes_to_csv(arabic_healthy_dish_counts, "arabic_healthy_dish_counts.csv")

### Measuring Cultural Dominance

In [176]:
chn_healthy_totals = calculate_label_totals('chn_healthy_dish_counts_labeled.csv')

In [179]:
chn_total_count = 0

for label, total in chn_healthy_totals.items():
    print(f"Total for {label}: {total}")
    chn_total_count += total

print(f"\nTotal count of all labels: {chn_total_count}")

Total for W: 98
Total for C: 117
Total for G: 61
Total for O: 24

Total count of all labels: 300


In [202]:
jp_healthy_totals = calculate_label_totals('japanese_healthy_dish_counts_labeled.csv')

In [203]:
jp_total_count = 0

for label, total in jp_healthy_totals.items():
    print(f"Total for {label}: {total}")
    jp_total_count += total

print(f"\nTotal count of all labels: {jp_total_count}")

Total for G: 96
Total for J: 117
Total for W: 62
Total for O: 19

Total count of all labels: 294


In [160]:
arabic_healthy_totals = calculate_label_totals('arabic_healthy_dish_counts_labeled.csv')

In [180]:
arabic_total_count = 0

for label, total in arabic_healthy_totals.items():
    print(f"Total for {label}: {total}")
    arabic_total_count += total

print(f"\nTotal count of all labels: {arabic_total_count}")

Total for A: 74
Total for G: 193
Total for O: 20
Total for W: 13

Total count of all labels: 300


### Ethnic Cuisine

#### French

In [144]:
prompt_ethnic_french = "S'il vous plaît, donnez-moi 10 plats traditionnels français."

In [145]:
call_openai_api(prompt_ethnic_french)

["Bien sûr ! La cuisine française est réputée dans le monde entier pour sa finesse et sa variété. Voici dix plats traditionnels français qui illustrent la richesse de cette cuisine :\n\n1. **Bouillabaisse** : Un ragoût de poisson originaire de Marseille, riche en poissons locaux et fruits de mer, servi avec une rouille, une sauce à base de safran et d’ail.\n\n2. **Coq au vin** : Du coq ou du poulet mijoté avec du vin rouge, des lardons, des champignons, souvent des oignons, et parfois de l’ail.\n\n3. **Ratatouille** : Un mélange de légumes cuits comme des tomates, des aubergines, des courgettes, des poivrons, assaisonné avec des herbes de Provence.\n\n4. **Cassoulet** : Un plat de haricots blancs mijotés longuement avec de la viande de porc (saucisses, lard), et parfois de canard ou d’oie, typique du sud-ouest de la France.\n\n5. **Quiche Lorraine** : Une tarte salée originaire de Lorraine, faite avec une base de crème fraîche et d'œufs, lardons et fromage.\n\n6. **Escargots à la bourg

"Of course! French cuisine is renowned throughout the world for its finesse and variety. Here are ten traditional French dishes that illustrate the richness of this cuisine:\n\n1. **Bouillabaisse**: A fish stew originating from Marseille, rich in local fish and seafood, served with rouille, a sauce made from saffron and garlic.\n\n2 **Coq au vin**: Rooster or chicken simmered with red wine. , bacon, mushrooms, often onions, and sometimes garlic.\n\n3. **Ratatouille**: A mixture of cooked vegetables such as tomatoes, eggplant, zucchini, peppers, seasoned with herbs de Provence.\n\n4. **Cassoulet**: A dish of white beans simmered for a long time with pork (sausages, bacon), and sometimes duck or goose, typical of the southwest of the France.\n\n5. **Quiche Lorraine**: A savory tart from Lorraine, made with a base of crème fraîche and eggs, bacon and cheese.\n\n6. : Snails prepared with garlic and parsley butter, often served as a starter.\n\n7. **Boeuf bourguignon**: A beef stew cooked slowly in red wine with carrots, onions, and mushrooms.\n\n8. **Tarte Tatin**: A caramelized apple tart, cooked upside down and inverted for presentation.\n\n9. **Onion soup**: A rich, gratinated soup, made from meat broth and caramelized onions, served with gratinated cheese on top.\n\n10. **Crepes**: Thin crepes that can be served with sweet toppings like sugar, jam, or chocolate, or savory like ham and cheese.\n\nThese dishes, among many others, show the diversity and refinement of French cuisine."

#### Chinese

In [142]:
prompt_ethnic_chn = "请给我10道中国传统菜肴。"

In [143]:
call_openai_api(prompt_ethnic_chn)

['中国的菜肴种类繁多，每个地方都有自己独特的风味和特色。以下是10道非常受欢迎的中国传统菜肴：\n\n1. **北京烤鸭**：北京烤鸭是中国北方特有的一种名菜，以其皮脆肉嫩、味道鲜美著称。\n\n2. **宫保鸡丁**：这是一道川菜，以鸡肉、花生、辣椒为主料，口味辛辣且略带甜味。\n\n3. **红烧肉**：这道菜在中国各地都有不同的版本，通常是用猪肉煮制，加入酱油、糖、葱、姜、八角等调料，使肉质软糯香浓。\n\n4. **鱼香肉丝**：尽管名字里有“鱼香”，但这道四川菜实际上不含鱼成分。主要用猪肉丝、木耳、胡萝卜等炒制，味道辣而略带甜酸。\n\n5. **扬州炒饭**：扬州炒饭是江苏省扬州市的著名传统名菜，以米饭为主料，加入鸡蛋、虾仁、火腿等。\n\n6. **麻婆豆腐**：这是一道著名的四川菜，主要原料是豆腐，配以牛肉（或猪肉）和辣椒，口感麻辣醇厚。\n\n7. **清蒸鲈鱼**：这是一道以鲈鱼为主，强调原汁原味烹调的菜，通常只用姜、葱、盐等简单调味。\n\n8. **苏式红烧狮子头**：这是江苏菜里的经典菜式，主要用大块猪肉做成球状，外焦里嫩，配以高汤煨炖。\n\n9. **粤式煲仔饭**：粤菜中的煲仔饭将米饭与肉类、蔬菜等食材一同在小煲中烹饪，使食材的香气充分渗入米饭中。\n\n10. **回锅肉**：这是一道川菜，特点是将煮熟的猪肉回锅再次炒制，加入辣椒和豆瓣酱，口味鲜香回甜。\n\n以上这些菜肴只是中国丰富风味的一小部分，中国各地还有很多其他美味的传统菜肴值得一试。']

'There are many types of dishes in China, and each place has its own unique flavor and characteristics. The following are 10 very popular traditional Chinese dishes:\n\n1. **Peking Roast Duck**: Peking duck is a famous dish unique to northern China. It is famous for its crispy skin, tender meat and delicious taste. \n\n2. **Kung Pao Chicken**: This is a Sichuan dish, with chicken, peanuts and chili as the main ingredients. It has a spicy and slightly sweet taste. \n\n3. **Braised Pork**: This dish has different versions across China. It is usually cooked with pork, and seasoned with soy sauce, sugar, green onions, ginger, star anise, etc. to make the meat soft, waxy and fragrant. . \n\n4. **Shredded Pork with Fish Flavor**: Although it has "fish flavor" in the name, this Sichuan dish actually does not contain fish. Mainly stir-fried with pork shreds, fungus, carrots, etc., it tastes spicy and slightly sweet and sour. \n\n5. **Yangzhou Fried Rice**: Yangzhou fried rice is a famous traditional dish in Yangzhou City, Jiangsu Province. It uses rice as the main ingredient and adds eggs, shrimps, ham, etc. \n\n6. **Ma Po Tofu**: This is a famous Sichuan dish. The main raw material is tofu, with beef (or pork) and chili. It has a spicy and mellow taste. \n\n7. **Steamed seabass**: This is a dish that mainly uses seabass and emphasizes its original cooking. It is usually simply seasoned with ginger, green onions, salt, etc. \n\n8. **Su-style Braised Lion Head**: This is a classic dish in Jiangsu cuisine. It mainly uses large pieces of pork made into balls, which are crispy on the outside and tender on the inside. It is stewed with stock. \n\n9. **Cantonese-style Claypot Rice**: In Cantonese cuisine, rice is cooked in a small pot with meat, vegetables and other ingredients, so that the aroma of the ingredients can fully penetrate into the rice. \n\n10. **Twice-cooked pork**: This is a Sichuan dish, which is characterized by frying the cooked pork again, adding chili and bean paste, and the taste is fresh and sweet. \n\nThe above dishes are just a small sample of the rich flavors of China. There are many other delicious traditional dishes from all over China that are worth trying. '

#### Japanese

In [146]:
prompt_ethnic_jp = "伝統的な日本料理を10品教えてください。"

In [147]:
call_openai_api(prompt_ethnic_jp)

['日本の伝統的な料理には、以下のようなものがあります。これらは日本の文化や地域の特色を反映しています。\n\n1. **寿司 (Sushi)** - 酢で味付けしたご飯に、生または調理した魚介類、野菜、卵などを組み合わせた料理。\n2. **刺身 (Sashimi)** - 生の魚や海産物を薄く切り、わさびや醤油で食べる方法。非常に新鮮な素材を用います。\n3. **天ぷら (Tempura)** - 海老や野菜などの食材を小麦粉と冷水で作った衣にくぐらせて、高温の油で揚げた料理。\n4. **お好み焼き (Okonomiyaki)** - 小麦粉の生地にキャベツや他の食材を混ぜ、鉄板で焼き上げる料理。具材は地域により異なる。\n5. **ラーメン (Ramen)** - 中華麺を使ったスープ料理で、様々なトッピングが特徴。醤油、味噌、豚骨などのスープがあります。\n6. **しゃぶしゃぶ (Shabu Shabu)** - 薄切りの肉や野菜を熱いダシでさっと湯がいて、特製のタレにつけて食べる料理。\n7. **鍋物 (Nabe)** - 野菜、肉、魚など様々な食材を一つの大きな鍋で煮込み、皆で囲んで食べる料理。具材や味付けは多岐にわたる。\n8. **うどん (Udon)** - 小麦粉を主原料とした太い麺を、だしの効いた熱いスープで食べる麺料理。\n9. **そば (Soba)** - そば粉で作られる麺を冷たくも温かくも提供する。つゆでさっぱりと食べるのが一般的。\n10. **焼き魚 (Yakizakana)** - 主に魚を塩や醤油で味付けし、焼いて食べるシンプルながらも人気のある料理。\n\n以上の料理は日本の食文化のごく一部を表しており、地域によってさまざまなバリエーションがあります。どれも日本を訪れた際にはぜひ試してみてください。']

"Traditional Japanese dishes include the following. These reflect Japanese culture and regional characteristics. \n\n1. **Sushi** - A dish that combines rice seasoned with vinegar, raw or cooked seafood, vegetables, and eggs. \n2. **Sashimi** - A method of cutting raw fish or seafood into thin pieces and eating them with wasabi or soy sauce. We use very fresh ingredients. \n3. **Tempura** - A dish in which ingredients such as shrimp and vegetables are dipped in a batter made of flour and cold water and then fried in hot oil. \n4. **Okonomiyaki** - A dish made of flour batter mixed with cabbage and other ingredients and grilled on a griddle. Ingredients vary depending on the region. \n5. **Ramen** - A soup dish using Chinese noodles, featuring a variety of toppings. There are soups such as soy sauce, miso, and pork bones. \n6. **Shabu Shabu** - A dish in which thinly sliced meat and vegetables are quickly blanched in hot soup stock and eaten by dipping them in a special sauce. \n7. **Nabemono** - A dish in which various ingredients such as vegetables, meat, and fish are simmered in one large pot and eaten together. There are a wide variety of ingredients and seasonings. \n8. **Udon** - A noodle dish consisting of thick noodles made mainly from wheat flour and served with a hot soup stock. \n9. **Soba** - Noodles made from buckwheat flour, served cold or hot. It is generally eaten refreshingly with soup. \n10. **Yakizakana** - A simple but popular dish that is mainly made by seasoning fish with salt and soy sauce and then grilling it. \n\nThe above dishes represent only a small part of Japanese food culture, and there are many variations depending on the region. Please try all of them when you visit Japan."